# 翼状胬肉诊断模型

# 准备

## 导入必要的库
导入PyTorch、OpenCV、Pandas等必要的库，为图像分类模型做准备。

In [ ]:
K = 5
base_seed = 420 # 用于 KFold 分割的随机种子

# --- 定义两阶段 Stacking 的基础 CNN 模型列表 ---
# Stage 1: 二分类 (正常 vs 有病)
CNN_FEATURE_EXTRACTORS_STAGE1 = ['ResNet18Classifier','EfficientNetB0Classifier', 'DenseNet121Classifier']
# Stage 2: 二分类 (观察 vs 手术) - 仅用于 Stage 1 预测为有病 (>0) 的情况
CNN_FEATURE_EXTRACTORS_STAGE2 = ['ResNet50Classifier','EfficientNetB4Classifier', 'DenseNet201Classifier']

# --- CNN 微调参数 (在每折内部使用) ---
cnn_micro_train_stage1_params = {
    'num_epochs': 25,    # 在每折中微调 CNN 的 epoch 数
    'lr': 4e-4,          # 微调的学习率
    'weight_decay': 2e-4,# 微调的权重衰减
    'patience': 5
}

cnn_micro_train_stage2_params = {
    'num_epochs': 35,
    'lr': 4e-4,
    'weight_decay': 2e-4,
    'patience': 7
}

# --- 元模型类型选择 ---
# 可选: 'LightGBM', 'LogisticRegression'
META_MODEL_TYPE = 'LightGBM'
# --- Logistic Regression 超参数 ---
logreg_params = {
    'solver': 'lbfgs',      # 求解器，lbfgs 通常适用于小数据集和多类问题
    'multi_class': 'auto',  # 多类问题策略，'auto' 会根据数据自动选择 'ovr' 或 'multinomial'
    'max_iter': 1000,       # 迭代次数，确保收敛
    'random_state': base_seed, # 随机种子
    'n_jobs': -1            # 使用所有可用核心
}
# --- LightGBM 超参数 ---
lgbm_params = {
    'objective': 'binary',     # 二分类任务
    'metric': 'binary_logloss',# Log Loss 作为评估指标
    'boosting_type': 'gbdt',   # 梯度提升决策树
    'n_estimators': 1000,      # 树的数量 (配合早停使用，可以设置得大一些)
    'learning_rate': 0.03,     # 学习率
    'num_leaves': 20,          # 控制树的复杂度，防止过拟合
    'max_depth': -1,           # 树的最大深度，-1表示不限制 (配合 num_leaves 控制)
    'seed': base_seed,         # 随机种子
    'n_jobs': -1,              # 使用所有可用核心
    'verbose': -1,             # 不打印中间信息
    'colsample_bytree': 0.7,   # 每棵树随机采样的特征比例
    'subsample': 0.7,          # 每棵树随机采样的样本比例
    'reg_alpha': 0.3,          # L1 正则化
    'reg_lambda': 0.3,         # L2 正则化
    'min_child_samples': 30    # 叶子节点的最小样本数
}

# ================== 缩放参数设置 =================
TARGET_SIZE = (512, 512)
TRAIN_SIZE = (512, 512)
output_format = "PNG" # 输出格式

# ================== 数据集路径 =================
# 数据路径
image_dir =          r"f:/train"
# colab路径
colab_zip_path = "/content/drive/My Drive/train.zip"
colab_extract_path = "/content/trains/"
# Kaggle路径
#kaggle_zip_path = "/kaggle/working/train.zip"
#kaggle_extract_path = "/kaggle/working/trains/"
kaggle_extract_path = "/kaggle/input/pterygium/train/"
kaggle_temp_path = "/kaggle/working/"

# =================== 验证集路径 =================
# 验证集路径
val_image_dir =      r"f:/val"
# colab路径
#colab_val_zip_path = "/content/drive/My Drive/val.zip"
#colab_val_extract_path = "/content/val/"
# Kaggle路径
kaggle_val_path = "/kaggle/input/pterygium/val_img/"

# =================== SHAP设置 =================
shap_scaling_factor = 100

In [ ]:
def setup_matplotlib_agg_backend_if_no_gui():
    """
    检查是否可能缺少 GUI 后端（例如，在无头服务器上运行）。
    如果是这种情况，将 Matplotlib 后端设置为 'Agg' 以避免错误。

    应该在首次导入 `matplotlib.pyplot` 之前调用此函数。
    """
    
    # 检查是否在非 Windows 系统上且没有设置 DISPLAY 环境变量
    # 这是判断是否缺少 GUI 的常见启发式方法
    try:
        # 尝试获取 IPython 实例
        shell = get_ipython().__class__.__name__ # type: ignore
        # 'ZMQInteractiveShell' 表示 Jupyter Notebook 或 QtConsole
        # 'TerminalInteractiveShell' 表示 IPython 命令行
        if 'Shell' in shell:
            # Jupyter/IPython 环境
            print('检测到jupyter环境')
            get_ipython().run_line_magic('matplotlib', 'inline') # type: ignore
            return True
        else:
            # 其他情况（理论上不应发生在此 try 块）
            raise NameError
    except NameError:
        print("检测到可能没有 GUI 环境，将 Matplotlib 后端设置为 'Agg'。")
        matplotlib.use('Agg') # type: ignore
        return False      # 标准 Python 解释器 (get_ipython 未定义)
    except Exception as e:
        print(f"警告：尝试将 Matplotlib 后端设置为 'Agg' 时出错: {e}")
        return False

In [ ]:
import random
import subprocess
import torch
import time
import gc
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms.functional import to_pil_image
import lightgbm as lgb
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Subset
import torch.backends.cudnn as cudnn
from torchvision import transforms, models
import pandas as pd
import boto3
import botocore
import shutil
import os
import zipfile
import shap
import sys
from PIL import Image
import platform
import numpy as np
import glob
from tqdm.autonotebook import tqdm # 好看！
import matplotlib
setup_matplotlib_agg_backend_if_no_gui()
import matplotlib.pyplot as plt
import matplotlib.font_manager

if platform.system() == "Windows":
    num_workers = 0
    print(f"检测到 Windows 系统，将 DataLoader 的 num_workers 设置为 {num_workers}。")
else:
    # 在非 Windows 系统（如 Linux/Colab）上
    num_workers = 4
    print(f"检测到非 Windows 系统 ({platform.system()})，将 DataLoader 的 num_workers 设置为 {num_workers}。")
    # 设置中文字体
    if not os.path.exists('simhei.ttf'):
        subprocess.run(['wget','-q','-O', 'simhei.ttf', "https://cdn.jsdelivr.net/gh/Haixing-Hu/latex-chinese-fonts/chinese/%E9%BB%91%E4%BD%93/SimHei.ttf"], check=True)
    matplotlib.font_manager.fontManager.addfont('simhei.ttf')
    matplotlib.rc('font', family='SimHei')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 配置GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"CUDA 可用: {torch.cuda.is_available()}")
print(f"使用的设备: {device}")

if torch.cuda.is_available():
    cudnn.benchmark = True
    print("cuDNN benchmark 模式已启用")

## 读取和准备数据
从train_classification_label.xlsx读取标签数据，并组织预处理后的图像数据路径。标签包括：0（健康）、1（建议观察）、2（建议手术）。

In [ ]:
if os.path.exists('.env'):
    from dotenv import load_dotenv
    load_dotenv('.env')

R2_ACCESS_KEY_ID = os.environ.get('R2_ACCESS_KEY_ID', '')
R2_SECRET_ACCESS_KEY = os.environ.get('R2_SECRET_ACCESS_KEY', '')
R2_BUCKET_NAME = os.environ.get('R2_BUCKET_NAME', '')
R2_ENDPOINT_URL = os.environ.get('R2_ENDPOINT_URL', '')

# 如果在云端上运行，从 Google Drive 读取数据
if 'google.colab' in sys.modules or os.path.exists("/kaggle/working"):
    if 'google.colab' in sys.modules:
        print('在 Google Colab 环境中运行')
        image_dir = os.path.join(colab_extract_path,"train")
        label_file = os.path.join(image_dir,"train_classification_label.xlsx")
        zip_path = colab_zip_path
        extract_path = colab_extract_path

        # Mount Google Drive
        from google.colab import drive # type: ignore
        from google.colab import userdata # type: ignore
        drive.mount('/content/drive')
        R2_ACCESS_KEY_ID = userdata.get("R2_ACCESS_KEY_ID")
        R2_SECRET_ACCESS_KEY = userdata.get("R2_SECRET_ACCESS_KEY")
        R2_BUCKET_NAME = userdata.get("R2_BUCKET_NAME")
        R2_ENDPOINT_URL = userdata.get("R2_ENDPOINT_URL")
    else:
        print('在 Kaggle 环境中运行')
        # Kaggle 环境下的路径设置
        # image_dir = os.path.join(kaggle_extract_path,"train")
        # label_file = os.path.join(image_dir,"train_classification_label.xlsx")
        # zip_path = kaggle_zip_path
        # extract_path = kaggle_extract_path

        # Google Drive 有每日下载次数限制，可能会导致下载失败
        # if not os.path.exists(zip_path):
        #     from kaggle_secrets import UserSecretsClient
        #     user_secrets = UserSecretsClient()
        #     !gdown --id {user_secrets.get_secret("train_zip_downloadurl")}
        image_dir = os.path.join(kaggle_extract_path,"train")
        label_file = os.path.join(image_dir,"train_classification_label.xlsx")
        val_image_dir = os.path.join(kaggle_val_path,"val_img")

        from kaggle_secrets import UserSecretsClient # type: ignore
        user_secrets = UserSecretsClient()
        R2_ACCESS_KEY_ID = user_secrets.get_secret("R2_ACCESS_KEY_ID")
        R2_SECRET_ACCESS_KEY = user_secrets.get_secret("R2_SECRET_ACCESS_KEY")
        R2_BUCKET_NAME = user_secrets.get_secret("R2_BUCKET_NAME")
        R2_ENDPOINT_URL = user_secrets.get_secret("R2_ENDPOINT_URL")

    if not os.path.exists(label_file):
        # 解压数据
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)    
else:
    print(f'不在云端环境中运行,使用本地数据路径{image_dir}')
label_file = os.path.join(image_dir,"train_classification_label.xlsx")

In [ ]:
# 自定义数据集类，用于读取图像和标签，并支持阶段性标签映射和过滤
class PterygiumDataset(Dataset):
    def __init__(self, initial_labels_df, image_dir, transform=None, stage=None):
        """
        初始化数据集
        :param initial_labels_df: 包含图像原始标签的 Pandas DataFrame
        :param image_dir: 图像文件夹路径 (应是resize后的图像路径)
        :param transform: 图像变换操作
        :param stage: 指定数据集用于哪个阶段 ('stage1', 'stage2')。
                    'stage1': 标签 0->0, 1->1, 2->1 (二分类: 正常 vs 有病)
                    'stage2': 过滤原标签 1, 2 的数据, 标签 1->0, 2->1 (二分类: 建议观察 vs 建议手术)
                    None: 标签保持原样 0, 1, 2 (三分类)
        """
        self.original_labels_df = initial_labels_df.copy()
        self.image_dir = image_dir
        self.transform = transform
        self.stage = stage
        
        # 根据阶段过滤和映射标签
        if self.stage == 'stage1':
            self.labels_df = self.original_labels_df.copy()
            # 阶段1标签映射: 0 -> 0, 1 -> 1, 2 -> 1
            self.labels_df['Pterygium'] = self.labels_df['Pterygium'].apply(lambda x: 0 if x == 0 else 1)
            print(f"数据集初始化为 Stage 1 (正常 vs 有病), 标签映射 0->0, >0->1. 数据量: {len(self.labels_df)}")
            print(f"阶段 1 标签分布: \n{self.labels_df['Pterygium'].value_counts()}")
        elif self.stage == 'stage2':
            # 阶段2数据过滤: 只保留原标签为 1 或 2 的数据
            self.labels_df = self.original_labels_df[self.original_labels_df['Pterygium'].isin([1, 2])].copy()
            # 阶段2标签映射: 1 -> 0, 2 -> 1
            self.labels_df['Pterygium'] = self.labels_df['Pterygium'].apply(lambda x: 0 if x == 1 else 1)
            print(f"数据集初始化为 Stage 2 (建议观察 vs 建议手术), 过滤原标签 0 数据. 数据量: {len(self.labels_df)}")
            print(f"阶段 2 标签映射 1->0, 2->1. 阶段 2 标签分布: \n{self.labels_df['Pterygium'].value_counts()}")
        else: # Default: original 3-class labels
            self.labels_df = self.original_labels_df.copy()
            print(f"数据集初始化为原始三分类 (0, 1, 2). 数据量: {len(self.labels_df)}")
            print(f"原始标签分布: \n{self.labels_df['Pterygium'].value_counts()}")

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        """
        获取指定索引的图像和标签 (已根据阶段进行映射)
        """
        row = self.labels_df.iloc[idx]
        image_name = row['Image']
        label = row['Pterygium'] # Use the potentially mapped label

        # 图像文件路径构建 (假设resize后的图像保存在 image_dir/0001/0001.png 或 image_dir/0001.png)
        # resize_and_save_image 函数对于训练集是保存在子文件夹，验证集是直接保存在根目录
        image_folder_name = f"{int(image_name):04d}"
        image_path_in_folder = os.path.join(self.image_dir, image_folder_name, f"{image_folder_name}.{output_format.lower()}")
        image_path_in_root = os.path.join(self.image_dir, f"{image_folder_name}.{output_format.lower()}")

        if os.path.exists(image_path_in_folder):
            image_path = image_path_in_folder
        elif os.path.exists(image_path_in_root):
            image_path = image_path_in_root
        else:
            raise FileNotFoundError(f"Image file not found for {image_name} at expected paths: {image_path_in_folder} or {image_path_in_root}")

        # 加载图像
        image = Image.open(image_path).convert("RGB")

        # 应用图像变换
        if self.transform:
            image = self.transform(image)

        return image, label

## 数据 Resize
只在Linux运行时使用，因为windows仅用与测试。

### 准备R2

In [ ]:
def create_r2_client():
    """尝试创建并返回一个配置好的 boto3 R2 客户端。"""
    # 确认环境变量已加载 (这些变量应在之前的单元格中设置)
    required_vars = ['R2_ENDPOINT_URL', 'R2_ACCESS_KEY_ID', 'R2_SECRET_ACCESS_KEY', 'R2_BUCKET_NAME']
    if not all(var in globals() and globals()[var] for var in required_vars):
        print("R2 配置不完整（缺少 Endpoint URL, Access Key, Secret Key 或 Bucket Name）。跳过 R2 缓存。")
        return None, False # 返回 None 和 R2 未配置标志

    global r2_configured # 声明我们要修改全局变量
    r2_configured = True # 标记 R2 已配置

    try:
        print("正在创建 R2 (boto3 S3) 客户端...")
        s3_client = boto3.client(
            service_name='s3',
            endpoint_url=R2_ENDPOINT_URL,
            aws_access_key_id=R2_ACCESS_KEY_ID,
            aws_secret_access_key=R2_SECRET_ACCESS_KEY,
            region_name='auto', # R2 通常使用 'auto'
            config=botocore.config.Config(signature_version='s3v4') # 明确签名版本
        )
        # 尝试列出 buckets (可选，作为连接测试)
        # s3_client.list_buckets()
        print("R2 客户端创建成功。")
        return s3_client, True
    except Exception as e:
        print(f"创建 R2 客户端时出错: {e}")
        r2_configured = False # 出错则标记为未配置
        return None, False

def check_r2_cache(s3_client, bucket_name, cache_key):
    """检查指定的缓存键是否存在于 R2 存储桶中。"""
    if not s3_client: return False
    try:
        s3_client.head_object(Bucket=bucket_name, Key=cache_key)
        return True
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == '404':
            return False # 文件未找到
        else:
            # 其他错误 (如权限问题)
            print(f"检查 R2 缓存时出错 (Key: {cache_key}): {e}")
            return False
    except Exception as e:
        print(f"检查 R2 缓存时发生未知错误: {e}")
        return False

def download_from_r2(s3_client, bucket_name, cache_key, local_path):
    """从 R2 下载文件到本地路径，带进度条。"""
    if not s3_client: return False
    try:
        # 获取文件大小以显示进度
        response = s3_client.head_object(Bucket=bucket_name, Key=cache_key)
        total_size = int(response.get('ContentLength', 0))

        print(f"正在从 R2 下载 {cache_key} 到 {local_path} ({total_size / (1024*1024):.2f} MB)...")
        with tqdm(total=total_size, unit='B', unit_scale=True, desc=cache_key, leave=False) as pbar:
            s3_client.download_file(
                Bucket=bucket_name,
                Key=cache_key,
                Filename=local_path,
                Callback=lambda bytes_transferred: pbar.update(bytes_transferred)
            )
        print(f"文件 {cache_key} 下载完成。")
        return True
    except botocore.exceptions.ClientError as e:
        print(f"从 R2 下载文件时出错 (Key: {cache_key}): {e}")
        # 如果文件下载失败，尝试删除本地可能不完整的文件
        if os.path.exists(local_path):
            try: os.remove(local_path)
            except: pass
        return False
    except Exception as e:
        print(f"下载 R2 文件时发生未知错误: {e}")
        if os.path.exists(local_path):
            try: os.remove(local_path)
            except: pass
        return False

def upload_to_r2(s3_client, bucket_name, local_path, cache_key):
    """将本地文件上传到 R2，带进度条。"""
    if not s3_client or not os.path.exists(local_path):
        print(f"上传 R2 失败：客户端未初始化或本地文件不存在 ({local_path})。")
        return False
    try:
        total_size = os.path.getsize(local_path)
        print(f"正在上传 {local_path} ({total_size / (1024*1024):.2f} MB) 到 R2 作为 {cache_key}...")
        with tqdm(total=total_size, unit='B', unit_scale=True, desc=cache_key, leave=False) as pbar:
            s3_client.upload_file(
                Filename=local_path,
                Bucket=bucket_name,
                Key=cache_key,
                Callback=lambda bytes_transferred: pbar.update(bytes_transferred)
            )
        print(f"文件 {cache_key} 上传完成。")
        return True
    except botocore.exceptions.ClientError as e:
        print(f"上传文件到 R2 时出错 (Key: {cache_key}): {e}")
        return False
    except Exception as e:
        print(f"上传 R2 文件时发生未知错误: {e}")
        return False

def zip_directory(folder_path, zip_path):
    """压缩指定文件夹的内容到 zip 文件。"""
    if not os.path.isdir(folder_path):
        print(f"错误：要压缩的文件夹不存在 {folder_path}")
        return False
    print(f"正在压缩目录 {folder_path} 到 {zip_path}...")
    try:
        with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            # 获取文件夹内的所有文件和子文件夹
            file_paths = []
            for root, dirs, files in os.walk(folder_path):
                for filename in files:
                    file_paths.append(os.path.join(root, filename))

            # 使用 tqdm 显示压缩进度 (按文件数)
            with tqdm(total=len(file_paths), desc="压缩文件", unit="file", leave=False) as pbar:
                for file in file_paths:
                    # 计算文件在 zip 中的相对路径
                    arcname = os.path.relpath(file, folder_path)
                    zipf.write(file, arcname)
                    pbar.update(1)
        print("目录压缩完成。")
        return True
    except Exception as e:
        print(f"压缩目录时出错: {e}")
        # 如果压缩失败，删除可能不完整的 zip 文件
        if os.path.exists(zip_path):
            try: os.remove(zip_path)
            except: pass
        return False

def unzip_directory(zip_path, extract_to_folder):
    """解压缩 zip 文件到指定文件夹。"""
    if not os.path.exists(zip_path):
        print(f"错误：要解压的 zip 文件不存在 {zip_path}")
        return False
    print(f"正在解压缩文件 {zip_path} 到 {extract_to_folder}...")
    try:
        os.makedirs(extract_to_folder, exist_ok=True) # 确保目标文件夹存在
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            # 获取 zip 文件中的成员数量以显示进度
            total_files = len(zip_ref.namelist())
            with tqdm(total=total_files, desc="解压缩文件", unit="file", leave=False) as pbar:
                # 使用 extractall 并更新进度条可能不直接，改为逐个提取
                for member in zip_ref.infolist():
                    zip_ref.extract(member, extract_to_folder)
                    pbar.update(1)
                    # 或者直接用 extractall，进度条可能不准确但更快
                    # zip_ref.extractall(extract_to_folder)
        print("文件解压缩完成。")
        return True
    except Exception as e:
        print(f"解压缩文件时出错: {e}")
        # 如果解压失败，可以选择是否删除不完整的解压目录
        # if os.path.exists(extract_to_folder):
        #     shutil.rmtree(extract_to_folder)
        return False

In [ ]:
resize_transform = transforms.Resize(TARGET_SIZE, interpolation=transforms.InterpolationMode.BILINEAR, antialias=True)

# --- Processing Function ---
def resize_and_save_image(img_info, base_input_dir, base_output_dir, transform, device):
    """
    Reads an image, resizes it (potentially on GPU), and saves it.
    """
    try:
        image_name = img_info['Image']
        image_name = f"{int(image_name):04d}"
        if os.path.exists(os.path.join(base_input_dir, f"{image_name}.png")):
            # 验证集图像路径
            input_path = os.path.join(base_input_dir, f"{image_name}.png")
            os.makedirs(base_output_dir, exist_ok=True)
            output_path = os.path.join(base_output_dir, f"{image_name}.{output_format.lower()}")
        else:
            # 训练集图像路径
            input_path = os.path.join(base_input_dir, image_name, f"{image_name}.png")
            # Create corresponding output subdirectory if it doesn't exist
            output_folder_path = os.path.join(base_output_dir, image_name)
            os.makedirs(output_folder_path, exist_ok=True)
            output_path = os.path.join(output_folder_path, f"{image_name}.{output_format.lower()}")

        # 1. Read image using PIL (CPU)
        img_pil = Image.open(input_path).convert("RGB")

        # 2. Convert PIL image to Tensor (CPU, scales to [0, 1])
        img_tensor_cpu = transforms.functional.to_tensor(img_pil) # Output: CxHxW

        # 3. Move tensor to GPU (if available)
        img_tensor_gpu = img_tensor_cpu.to(device)

        # 4. Apply Resize transform (GPU)
        resized_tensor_gpu = transform(img_tensor_gpu)

        # 5. Move resized tensor back to CPU
        resized_tensor_cpu = resized_tensor_gpu.cpu()

        # 6. Convert tensor back to PIL Image (CPU)
        # to_pil_image expects CxHxW tensor in [0, 1] range
        resized_img_pil = to_pil_image(resized_tensor_cpu)

        # 7. Save the resized PIL image (CPU)
        resized_img_pil.save(output_path, format=output_format)
        
        return True # Indicate success

    except FileNotFoundError:
        print(f"错误: 文件未找到 {input_path}")
        return False
    except Exception as e:
        print(f"错误处理图像 {input_path}: {e}")
        return False

In [ ]:
if 'google.colab' in sys.modules:
    print('在 Google Colab 环境中运行')
    original_image_dir = os.path.join(colab_extract_path,"train")
    output_dir = os.path.join(colab_extract_path,"train_resized")
    temp_dir = colab_extract_path
elif os.path.exists("/kaggle/working"):
    print('在 Kaggle 环境中运行')
    original_image_dir = os.path.join(kaggle_extract_path,"train")
    output_dir = os.path.join(kaggle_temp_path,"train_resized")
    temp_dir = kaggle_temp_path
else:
    print("错误: 无法识别的非 Windows 环境（可能是Linux），需要手动处理")
    exit(1)

if original_image_dir:
    print(f"原始输入目录: {original_image_dir}")
    print(f"目标输出目录: {output_dir}")
    print(f"临时文件目录: {temp_dir}")
    print(f"目标尺寸: {TARGET_SIZE}")

    # 创建 R2 客户端并检查配置
    s3_client, r2_configured = create_r2_client()
    r2_cache_key = f"work1_resized_{TARGET_SIZE[0]}x{TARGET_SIZE[1]}.zip"
    print(f"生成的 R2 缓存键: {r2_cache_key}")
    r2_local_zip_path = os.path.join(temp_dir, r2_cache_key)
    resize_done = False
    if os.path.exists(output_dir) and os.listdir(output_dir):
        print("检测到已存在的resize数据在本地，跳过resize步骤")
        resize_done = True

    # --- If not found locally, try R2 cache ---
    os.makedirs(output_dir, exist_ok=True)
    if not resize_done and r2_configured:
        print(f"本地目录 {output_dir} 为空或不存在，尝试检查 R2 缓存...")
        if check_r2_cache(s3_client, R2_BUCKET_NAME, r2_cache_key):
            print(f"检测到 R2 缓存文件: {r2_cache_key}. 尝试下载...")
            # Download the cache
            if download_from_r2(s3_client, R2_BUCKET_NAME, r2_cache_key, r2_local_zip_path):
                print(f"R2 缓存下载成功。正在解压到 {output_dir}...")
                # Unzip the cache
                # Ensure output_dir is clean before extracting to avoid mixing old/new files
                if os.path.exists(output_dir):
                    try: shutil.rmtree(output_dir)
                    except Exception as e: print(f"警告: 清理旧的输出目录失败: {e}")
                os.makedirs(output_dir, exist_ok=True) # Recreate empty directory
                if unzip_directory(r2_local_zip_path, output_dir):
                    print("R2 缓存解压成功。跳过本地resize步骤。")
                    resize_done = True # Data loaded from R2 cache
                    # Clean up the temporary zip file after extraction
                    if os.path.exists(r2_local_zip_path):
                        try: os.remove(r2_local_zip_path)
                        except Exception as e: print(f"警告: 清理本地zip文件失败: {e}")
                else:
                    print("错误: R2 缓存解压失败。将执行本地resize。")
                    resize_done = False # Reset flag to perform local resize
                    # Clean up potentially incomplete extraction directory
                    if os.path.exists(output_dir):
                        try: shutil.rmtree(output_dir)
                        except Exception as e: print(f"警告: 清理不完整输出目录失败: {e}")
            else:
                print("错误: 从 R2 下载缓存失败。将执行本地resize。")
                resize_done = False # Reset flag to perform local resize
        else:
            print("未检测到 R2 缓存文件。将执行本地resize。")
            resize_done = False # Ensure flag is false
    elif not resize_done and not r2_configured:
        print("R2 未配置或初始化失败，将执行本地resize。")
        resize_done = False # Ensure flag is false
    # --- Perform local resizing if not done by cache ---
    if not resize_done:
        print("执行本地图像resize...")
        try:
            labels_df = pd.read_excel(label_file)
        except Exception as e:
            print(f"Error reading label file {label_file}: {e}")
            sys.exit(1)
        success_count = 0
        error_count = 0
        # Create the main output directory BEFORE starting the loop (done above, but ensure it exists)
        os.makedirs(output_dir, exist_ok=True)
        # Iterate through images listed in the label file
        # Use original_image_dir as input base for resizing
        for index, row in tqdm(labels_df.iterrows(), total=len(labels_df), desc="Resizing Images"):
            if resize_and_save_image(row, original_image_dir, output_dir, resize_transform, device):
                success_count += 1
            else:
                error_count += 1
        print(f"\n本地处理完成!")
        print(f"成功处理图像数: {success_count}")
        print(f"处理失败图像数: {error_count}")
        print(f"处理后的图像保存在: {output_dir}")
        # --- Upload resized data to R2 cache if configured and resizing was successful ---
        if r2_configured and success_count > 0: # Only upload if some files were processed successfully
            print(f"将本地resize后的数据上传到 R2 缓存 ({r2_cache_key})...")
            # Create a zip file of the output directory
            if zip_directory(output_dir, r2_local_zip_path):
                # Upload the zip file
                if upload_to_r2(s3_client, R2_BUCKET_NAME, r2_local_zip_path, r2_cache_key):
                    print("R2 缓存上传成功。")
                else:
                    print("错误: R2 缓存上传失败。")
                # Clean up the temporary local zip file after upload attempt
                if os.path.exists(r2_local_zip_path):
                    try: os.remove(r2_local_zip_path)
                    except Exception as e: print(f"警告: 清理本地zip文件失败: {e}")
            else:
                print("错误: 创建本地 zip 文件失败，跳过 R2 上传。")
        elif not r2_configured:
            print("R2 未配置，跳过上传resize后的数据。")
        elif success_count == 0:
            print("本地resize失败（成功处理图像数为0），跳过R2上传。")
    image_dir = output_dir
else:
    print("未识别的非 Windows 环境，跳过图片resize步骤。")
    pass

## 创建数据加载器
使用PyTorch的Dataset和DataLoader类创建数据集和加载器，包括数据增强和训练/验证集的划分。

### 模拟高光的数据增强策略

In [ ]:
from PIL import Image, ImageDraw
import random
import torchvision.transforms.functional as TF

class AddRandomHighlight:
    """
    一个 torchvision transform，用于在 PIL 图像上随机添加圆形高光。

    参数:
        p (float): 应用此变换的概率 (0 到 1)。
        max_highlights (int): 单张图像上添加的最大高光数量（实际数量将在1到max_highlights之间随机选择）。
        radius_range (tuple): 一个包含两个整数的元组 (min_radius, max_radius)，指定高光圆形的半径范围。
        color (tuple): 一个包含三个整数的元组 (R, G, B)，指定高光的颜色 (默认为白色)。
    """
    def __init__(self, p=0.5, max_highlights=3, radius_range=(5, 15), color=(255, 255, 255)):
        if not 0.0 <= p <= 1.0:
            raise ValueError(f"概率 p 必须在 [0, 1] 范围内, 但得到 {p}")
        if not (isinstance(max_highlights, int) and max_highlights >= 1):
            raise ValueError(f"最大高光数 max_highlights 必须是 >= 1 的整数, 但得到 {max_highlights}")
        if not (isinstance(radius_range, tuple) and len(radius_range) == 2 and
                isinstance(radius_range[0], int) and isinstance(radius_range[1], int) and
                0 < radius_range[0] <= radius_range[1]):
            raise ValueError(f"半径范围 radius_range 必须是 (min, max) 形式的正整数元组，且 min <= max, 但得到 {radius_range}")
        if not (isinstance(color, tuple) and len(color) == 3 and all(0 <= c <= 255 for c in color)):
            raise ValueError(f"颜色 color 必须是 (R, G, B) 形式的元组，且值在 [0, 255] 范围内, 但得到 {color}")
            
        self.p = p
        self.max_highlights = max_highlights
        self.radius_range = radius_range
        self.color = color

    def __call__(self, img):
        """
        对输入的 PIL 图像应用变换。

        参数:
            img (PIL.Image.Image): 输入的 PIL 图像。

        返回:
            PIL.Image.Image: 可能添加了高光的 PIL 图像。
        """
        # 以概率 p 应用此变换
        if random.random() < self.p:
            # 复制图像以避免修改原始图像（如果原始图像后续还需使用）
            # 如果这是 Compose 链中的一步，通常不需要显式复制
            # img = img.copy() # 如果需要确保不修改原始输入，取消注释此行

            # 随机决定生成多少个高光 (至少1个)
            # 根据用户要求“小于四个”，我们生成 1 到 max_highlights (这里是3) 个
            num_highlights = random.randint(1, self.max_highlights)
            
            # 获取图像尺寸
            width, height = img.size
            
            # 创建 ImageDraw 对象以在图像上绘制
            draw = ImageDraw.Draw(img)

            for _ in range(num_highlights):
                # 随机选择半径
                radius = random.randint(self.radius_range[0], self.radius_range[1])
                
                # 随机选择圆心位置
                # 确保圆心位置加上半径不会超出图像边界太多（允许部分圆在边缘）
                # 稍微限制圆心范围，避免完全生成在图像外的圆心
                center_x = random.randint(0, width - 1) 
                center_y = random.randint(0, height - 1)

                # 计算圆形的边界框 (left, top, right, bottom)
                # ellipse 方法绘制的是指定边界框内的椭圆，如果边界框是正方形，则为圆形
                left = center_x - radius
                top = center_y - radius
                right = center_x + radius
                bottom = center_y + radius
                
                # 绘制实心圆形高光
                draw.ellipse([left, top, right, bottom], fill=self.color)
            
        # 返回处理后的图像（可能是原始图像或添加了高光的图像）
        return img

    def __repr__(self):
        # 提供一个清晰的表示形式，方便调试
        return f"{self.__class__.__name__}(p={self.p}, max_highlights={self.max_highlights}, radius_range={self.radius_range}, color={self.color})"

### 应用数据增强

In [ ]:
# 数据变换
train_transform = transforms.Compose([
    transforms.Resize((int(TRAIN_SIZE[0]*1.2), int(TRAIN_SIZE[1]*1.2))), # 先放大一点
    transforms.RandomCrop(TRAIN_SIZE), # 随机裁剪回目标尺寸
    transforms.RandomHorizontalFlip(p=0.5), # 随机水平翻转
    transforms.RandomRotation(degrees=20), # 随机旋转
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1), # 随机颜色抖动
    AddRandomHighlight(p=0.3, max_highlights=3, radius_range=(5, 12)), # 添加高光抑制模型关注高光问题
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 定义验证集/测试集的变换
val_transform = transforms.Compose([
    transforms.Resize(TRAIN_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 划分训练集和验证集，并创建对应的数据加载器
from sklearn.model_selection import train_test_split

# 读取标签文件
labels_df = pd.read_excel(label_file)

# 按照8:2的比例划分训练集和验证集
train_df, val_df = train_test_split(labels_df, test_size=0.2, random_state=420, stratify=labels_df['Pterygium'])

# 读取标签文件
if 'label_file' not in globals() or not os.path.exists(label_file):
    print("错误: 标签文件路径未设置或文件不存在。请检查前面的数据准备步骤。")
    sys.exit("标签文件未找到，无法继续。")
full_labels_df = pd.read_excel(label_file)

# 过滤掉标签不是 0, 1, 2 的数据（如果存在的话）
full_labels_df = full_labels_df[full_labels_df['Pterygium'].isin([0, 1, 2])].reset_index(drop=True)

# 创建基础数据集实例，不应用变换 (变换在创建 Subset 时按需应用)
# 这个数据集将用于根据索引创建 Subset
base_full_dataset_no_transform = PterygiumDataset(initial_labels_df=full_labels_df, image_dir=image_dir, transform=None,stage=None)

print(f"成功加载完整数据集，共 {len(base_full_dataset_no_transform)} 张图像。")
# 存储原始标签，用于 Stage 1 K-Fold split 和最终评估
all_original_labels = base_full_dataset_no_transform.labels_df['Pterygium'].values
all_original_indices = base_full_dataset_no_transform.labels_df.index.values # 存储DataFrame的索引，用于后续映射

# 准备 Stage 1 标签 (0: 正常, 1: 有病)
# 映射规则: 0 -> 0, 1 -> 1, 2 -> 1
stage1_labels = np.where(all_original_labels > 0, 1, 0)

# 准备 Stage 2 数据和标签 (仅限原始标签 1 或 2 的数据)
stage2_indices = np.where(all_original_labels > 0)[0] # 找到原始标签 > 0 的索引
stage2_original_labels = all_original_labels[stage2_indices] # 对应的原始标签 (1或2)
# Note: Stage 2 labels will be the original 1 and 2.
# We might map them to 0 and 1 internally for Stage 2 model training,
# but need to map back for combined prediction. Let's keep them as 1 and 2
# and set num_class=2 in lgbm_params, relying on LightGBM/LogReg to handle these labels.
# Or, map 1->0, 2->1 for training and then map back 0->1, 1->2 for final prediction.
# Let's map 1->0, 2->1 for training Stage 2 meta-model for consistency with 0-based indexing.
stage2_train_labels_mapped = np.where(stage2_original_labels == 1, 0, 1) # Map 1 -> 0, 2 -> 1 for Stage 2 training

print(f"Stage 1 数据量: {len(all_original_labels)} ({np.sum(stage1_labels == 0)} 正常, {np.sum(stage1_labels == 1)} 有病)")
print(f"Stage 2 (有病) 数据量: {len(stage2_indices)} ({np.sum(stage2_original_labels == 1)} 观察, {np.sum(stage2_original_labels == 2)} 手术)")


# 保存划分后的数据集到文件
train_label_file = os.path.join(image_dir, "train_classification_label_train.xlsx")
val_label_file = os.path.join(image_dir, "train_classification_label_val.xlsx")
if os.path.exists("/kaggle/working"):
    train_label_file = os.path.join(kaggle_temp_path, "train_classification_label_train.xlsx")
    val_label_file = os.path.join(kaggle_temp_path, "train_classification_label_val.xlsx")
train_df.to_excel(train_label_file, index=False)
val_df.to_excel(val_label_file, index=False)

# 创建训练集和验证集的数据集对象 (使用不同的 transform)
train_dataset = PterygiumDataset(initial_labels_df=train_df, image_dir=image_dir, transform=train_transform) # 使用训练变换
val_dataset = PterygiumDataset(initial_labels_df=val_df, image_dir=image_dir, transform=val_transform) # 使用验证变换

# 构建模型

## 构建 ResNet 模型
使用PyTorch的预训练ResNet18模型，修改最后的全连接层以适应3个类别的分类任务。

In [ ]:
from torchvision.models import ResNet18_Weights, ResNet34_Weights, ResNet50_Weights, ResNet101_Weights, ResNet152_Weights

class ResNet18Classifier(nn.Module):
    def __init__(self, num_classes=3, dropout_rate=0.5):
        super().__init__()
        self.resnet18 = models.resnet18(weights=ResNet18_Weights.DEFAULT)
        in_features = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Sequential(
            nn.Dropout(p=dropout_rate),
            nn.Linear(in_features, num_classes)
        )
    def forward(self, x):
        return self.resnet18(x)

class ResNet34Classifier(nn.Module):
    def __init__(self, num_classes=3, dropout_rate=0.5):
        super().__init__()
        self.resnet34 = models.resnet34(weights=ResNet34_Weights.DEFAULT)
        in_features = self.resnet34.fc.in_features
        self.resnet34.fc = nn.Sequential(
            nn.Dropout(p=dropout_rate),
            nn.Linear(in_features, num_classes)
        )
    def forward(self, x):
        return self.resnet34(x)

class ResNet50Classifier(nn.Module):
    def __init__(self, num_classes=3, dropout_rate=0.5):
        super().__init__()
        self.resnet50 = models.resnet50(weights=ResNet50_Weights.DEFAULT)
        in_features = self.resnet50.fc.in_features
        self.resnet50.fc = nn.Sequential(
            nn.Dropout(p=dropout_rate),
            nn.Linear(in_features, num_classes)
        )
    def forward(self, x):
        return self.resnet50(x)
    
class ResNet101Classifier(nn.Module):
    def __init__(self, num_classes=3, dropout_rate=0.5):
        super().__init__()
        self.resnet101 = models.resnet101(weights=ResNet101_Weights.DEFAULT)
        in_features = self.resnet101.fc.in_features
        self.resnet101.fc = nn.Sequential(
            nn.Dropout(p=dropout_rate),
            nn.Linear(in_features, num_classes)
        )
    def forward(self, x):
        return self.resnet101(x)

class ResNet152Classifier(nn.Module):
    def __init__(self, num_classes=3, dropout_rate=0.5):
        super().__init__()
        self.resnet152 = models.resnet152(weights=ResNet152_Weights.DEFAULT)
        in_features = self.resnet152.fc.in_features
        self.resnet152.fc = nn.Sequential(
            nn.Dropout(p=dropout_rate),
            nn.Linear(in_features, num_classes)
        )
    def forward(self, x):
        return self.resnet152(x)

## 构建EfficientNet

In [ ]:
from torchvision.models import EfficientNet_B0_Weights, efficientnet_b0, efficientnet_b4, EfficientNet_B4_Weights

class EfficientNetB0Classifier(nn.Module):
    def __init__(self, num_classes=3, dropout_rate=0.5): # num_classes 将根据阶段设置为 2
        super().__init__()
        self.efficientnet = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)
        in_features = self.efficientnet.classifier[-1].in_features
        self.efficientnet.classifier = nn.Sequential(
            nn.Dropout(p=dropout_rate),
            nn.Linear(in_features, num_classes)
        )
    def forward(self, x):
        return self.efficientnet(x)

class EfficientNetB4Classifier(nn.Module):
    def __init__(self, num_classes=3, dropout_rate=0.5):
        super().__init__()
        # 加载 EfficientNetB4 预训练模型
        self.efficientnet = efficientnet_b4(weights=EfficientNet_B4_Weights.IMAGENET1K_V1)
        # EfficientNet 的分类器是一个 Sequential 模块 (Dropout, Linear)
        # 获取最终 Linear 层的输入特征数
        in_features = self.efficientnet.classifier[-1].in_features 
        # 替换整个分类器模块
        self.efficientnet.classifier = nn.Sequential(
            nn.Dropout(p=dropout_rate),
            nn.Linear(in_features, num_classes)
        )
    def forward(self, x):
        return self.efficientnet(x)

## 构建DenseNet

In [ ]:
from torchvision.models import densenet121, densenet201, DenseNet121_Weights, DenseNet201_Weights

class DenseNet121Classifier(nn.Module):
    def __init__(self, num_classes=3, dropout_rate=0.5):
        super().__init__()
        # 加载 DenseNet121 预训练模型
        self.densenet = densenet121(weights=DenseNet121_Weights.IMAGENET1K_V1)
        # DenseNet 的分类器是一个 Linear 层
        # 获取 Linear 层的输入特征数
        in_features = self.densenet.classifier.in_features
        # 替换分类器层 (为了与 ResNet 保持一致，使用 Sequential 包含 Dropout 和 Linear)
        self.densenet.classifier = nn.Sequential( 
            nn.Dropout(p=dropout_rate),
            nn.Linear(in_features, num_classes)
        )
    def forward(self, x):
        return self.densenet(x)
    
class DenseNet201Classifier(nn.Module):
    def __init__(self, num_classes=3, dropout_rate=0.5): # num_classes 将根据阶段设置为 2
        super().__init__()
        self.densenet = densenet201(weights=DenseNet201_Weights.IMAGENET1K_V1)
        in_features = self.densenet.classifier.in_features
        self.densenet.classifier = nn.Sequential(
            nn.Dropout(p=dropout_rate),
            nn.Linear(in_features, num_classes)
        )
    def forward(self, x):
        return self.densenet(x)

## 构建ConvNeXt Base

In [ ]:
from torchvision.models import convnext_base, ConvNeXt_Base_Weights
class ConvNeXtBaseClassifier(nn.Module):
    def __init__(self, num_classes=3, dropout_rate=0.5):
        super().__init__()
        # 加载 ConvNeXt Base 预训练模型
        self.convnext = convnext_base(weights=ConvNeXt_Base_Weights.IMAGENET1K_V1)

        # ConvNeXt 的分类器结构是 Sequential(LayerNorm2d, Flatten, Linear)
        # 我们需要替换最后的 Linear 层，并在其前面添加 Dropout
        original_classifier_layers = list(self.convnext.classifier.children())
        
        # 获取原始 Linear 层的输入特征数
        original_linear_layer = original_classifier_layers[-1]
        in_features = original_linear_layer.in_features
        
        # 构建新的分类器 Sequential: 保留之前的层 -> 添加 Dropout -> 添加新的 Linear 层
        new_classifier_layers = original_classifier_layers[:-1] # 保留 LayerNorm2d 和 Flatten
        new_classifier_layers.append(nn.Dropout(p=dropout_rate)) # 添加 Dropout
        new_classifier_layers.append(nn.Linear(in_features, num_classes)) # 添加新的 Linear 层

        # 替换原模型的分类器模块
        self.convnext.classifier = nn.Sequential(*new_classifier_layers)

    def forward(self, x):
        # 前向传播
        return self.convnext(x)

# 模型保存和加载

In [ ]:
import joblib

# 保存 Boosting 模型
def save_boosting_model(model, path):
    joblib.dump(model, path)
    print(f"Boosting 模型已保存到 {path}")

# 加载 Boosting 模型
def load_boosting_model(path):
    model = joblib.load(path)
    print(f"Boosting 模型已从 {path} 加载")
    return model

# 保存 CNN 模型参数 (如果需要在预测时加载 CNN 特征提取器，例如在最终预测阶段)
def save_cnn_state_dict(model, path):
    torch.save(model.state_dict(), path)
    print(f"CNN 模型参数已保存到 {path}")

# 加载 CNN 模型参数
def load_cnn_state_dict(model, path, device):
    model.load_state_dict(torch.load(path, map_location=device, weights_only=True))
    model = model.to(device)
    print(f"CNN 模型参数已从 {path} 加载")
    return model

# Stacked Ensemble K 折交叉验证 (两阶段)

In [ ]:
# --- 存储 K-Fold 结果 ---
# 用于收集所有折的 OOF 预测和标签
all_oof_indices_stage1 = []
all_oof_meta_features_stage1 = []
all_oof_labels_stage1 = [] # Stage 1 (0 vs 1) labels

all_oof_indices_stage2 = []
all_oof_meta_features_stage2 = []
all_oof_labels_stage2 = [] # Stage 2 (1 vs 2) original labels

# --- 创建 Stage 1 KFold 分割器 (基于 Stage 1 标签) ---
if K > 0 and len(stage1_labels) > 0:
    skf_stage1 = StratifiedKFold(n_splits=K, shuffle=True, random_state=base_seed)
    print(f"开始进行 Stage 1 ({K}-Fold Cross-Validation: 0 vs >0)...")
else:
    print("Skipping Stage 1 K-Fold Cross-Validation due to data preparation error or K=0.")
    K = 0 # Ensure K=0 if data is missing

# --- Stage 1 K-Fold 循环 ---
if K > 0:
    current_fold_num = 0
    # Iterate on the indices of the full dataset
    for train_idx_fold_stage1, val_idx_fold_stage1 in skf_stage1.split(np.arange(len(base_full_dataset_no_transform)), stage1_labels):
        current_fold_num += 1
        fold_id_str = f"Stage 1 - Fold {current_fold_num}/{K}"
        print(f"\n--- 开始 {fold_id_str} ---")

        # --- 1. 创建当前折的 Subset 数据集和 DataLoader ---
        # 使用原始数据集，但通过索引和变换创建 Subset
        train_dataset_fold_stage1 = Subset(base_full_dataset_no_transform, train_idx_fold_stage1)
        val_dataset_fold_stage1 = Subset(base_full_dataset_no_transform, val_idx_fold_stage1)
        
        # 为 Subset 应用变换
        train_dataset_fold_stage1.dataset.transform = train_transform
        val_dataset_fold_stage1.dataset.transform = val_transform

        train_loader_fold_stage1 = DataLoader(train_dataset_fold_stage1,
                                            batch_size=64 if TRAIN_SIZE[0] <257 else 30,
                                            shuffle=True,
                                            num_workers=num_workers,
                                            prefetch_factor=2 if platform.system() == "Windows" else 8,
                                            pin_memory=False)
        val_loader_fold_stage1 = DataLoader(val_dataset_fold_stage1,
                                            batch_size=64 if TRAIN_SIZE[0] <257 else 30,
                                            shuffle=False, # 验证集不 shuffle
                                            num_workers=num_workers,
                                            prefetch_factor=2 if platform.system() == "Windows" else 8,
                                            pin_memory=False)
        print(f"{fold_id_str}: Train size={len(train_dataset_fold_stage1)}, Val size={len(val_dataset_fold_stage1)}")

        # --- 2. 微调 Stage 1 CNN 基础分类器 ---
        print(f"--- {fold_id_str}: 微调 CNN 基础分类器 (Level-0 Models) ---")
        
        fold_cnn_finetuned_classifiers_stage1 = {} # 存储微调好的完整 CNN 分类器

        # CNN base models for Stage 1 predict num_classes=2 (0 or 1)
        num_classes_stage1_cnn = 2 
        
        for cnn_name in CNN_FEATURE_EXTRACTORS_STAGE1:
            print(f"  微调 {cnn_name} (Stage 1)...")
            # 创建一个新的 CNN 模型实例 (带分类头), 输出层适应 2 类
            cnn_model_for_finetuning_stage1 = globals()[cnn_name](num_classes=num_classes_stage1_cnn).to(device)
            
            best_val_loss = float('inf')
            patience_counter = 0
            best_model_state_dict = None
            
            cnn_optimizer = optim.AdamW(cnn_model_for_finetuning_stage1.parameters(), 
                                        lr=cnn_micro_train_stage1_params['lr'], 
                                        weight_decay=cnn_micro_train_stage1_params['weight_decay'])
            cnn_scheduler = optim.lr_scheduler.CosineAnnealingLR(cnn_optimizer, 
                                                                T_max=cnn_micro_train_stage1_params['num_epochs'], 
                                                                eta_min=1e-6)
            cnn_criterion = nn.CrossEntropyLoss()
            scaler_cnn = torch.amp.GradScaler('cuda') 
            
            start_time_cnn_finetune = time.time()
            
            for cnn_epoch in range(cnn_micro_train_stage1_params['num_epochs']):
                cnn_model_for_finetuning_stage1.train()
                train_loss_cnn = 0
                train_correct_cnn = 0
                train_total_cnn = 0
                
                cnn_train_loader_tqdm = tqdm(train_loader_fold_stage1, desc=f'  {cnn_name} Stage 1 Epoch {cnn_epoch+1}/{cnn_micro_train_stage1_params["num_epochs"]}', leave=False)
                
                for batch_idx, (inputs, targets_orig) in enumerate(cnn_train_loader_tqdm):
                    # Map original labels to Stage 1 labels (0 or 1) for CNN training
                    targets_stage1 = torch.where(targets_orig > 0, 1, 0).to(device)
                    inputs = inputs.to(device)
                    
                    cnn_optimizer.zero_grad()
                    with torch.amp.autocast('cuda'):
                        outputs = cnn_model_for_finetuning_stage1(inputs)
                        loss = cnn_criterion(outputs, targets_stage1) # Use Stage 1 labels for loss
                    scaler_cnn.scale(loss).backward()
                    scaler_cnn.step(cnn_optimizer)
                    scaler_cnn.update()
                    train_loss_cnn += loss.item()
                    _, predicted = outputs.max(1)
                    train_total_cnn += targets_stage1.size(0)
                    train_correct_cnn += predicted.eq(targets_stage1).sum().item()
                    
                    cnn_train_loader_tqdm.set_postfix({
                        'loss': f'{loss.item():.4f}',
                        'acc': f'{100. * train_correct_cnn / train_total_cnn:.2f}%',
                        'lr': f'{cnn_optimizer.param_groups[0]["lr"]:.1e}'
                    })
                cnn_scheduler.step() # Update CNN learning rate

                # --- 基于验证集损失的早停 ---
                cnn_model_for_finetuning_stage1.eval()
                val_loss_cnn = 0
                val_total_cnn = 0
                with torch.no_grad():
                    for inputs, targets_orig in val_loader_fold_stage1:
                        targets_stage1_val = torch.where(targets_orig > 0, 1, 0).to(device) # Use Stage 1 mapped labels for validation loss
                        inputs = inputs.to(device)
                        with torch.amp.autocast('cuda'):
                            outputs = cnn_model_for_finetuning_stage1(inputs)
                            loss = cnn_criterion(outputs, targets_stage1_val)
                        val_loss_cnn += loss.item()
                        val_total_cnn += targets_stage1_val.size(0)

                avg_val_loss = val_loss_cnn / len(val_loader_fold_stage1) if len(val_loader_fold_stage1) > 0 else float('inf')
                print(f"    {fold_id_str} {cnn_name} Stage 1 Epoch {cnn_epoch+1}: Train Loss {train_loss_cnn/len(train_loader_fold_stage1):.4f}, Val Loss {avg_val_loss:.4f}")

                # Initialize best_val_loss and best_model_state_dict before the epoch loop
                if 'best_val_loss' not in locals() or avg_val_loss < best_val_loss:
                    best_val_loss = avg_val_loss
                    patience_counter = 0 # Reset patience
                    best_model_state_dict = cnn_model_for_finetuning_stage1.state_dict() # Save best model state
                    print(f"    {fold_id_str} {cnn_name} Val Loss improved. Saving model state.")
                else:
                    patience_counter += 1 # Increment patience if no improvement
                    print(f"    {fold_id_str} {cnn_name} Val Loss did not improve. Patience: {patience_counter}/{cnn_micro_train_stage1_params['patience']}")

                cnn_model_for_finetuning_stage1.train() # Set model back to train mode

                if patience_counter >= cnn_micro_train_stage1_params['patience']:
                    print(f"\n    {fold_id_str} {cnn_name} Stage 1 Early stopping triggered at epoch {cnn_epoch+1}.")
                    break

            # --- 训练结束后，加载验证集上表现最好的模型状态 ---
            if best_model_state_dict is not None:
                cnn_model_for_finetuning_stage1.load_state_dict(best_model_state_dict)
                print(f"    {fold_id_str} {cnn_name} Stage 1: Loaded best model state based on validation loss.")
            else:
                print(f"    {fold_id_str} {cnn_name} Stage 1: No best model state saved (maybe due to inf loss?). Using last epoch model state.")

            end_time_cnn_finetune = time.time()
            print(f"  微调 {cnn_name} (Stage 1) 完成，耗时: {end_time_cnn_finetune - start_time_cnn_finetune:.2f} 秒")

            # --- 评估微调后的基础 CNN 分类器在验证集上的性能 (for reporting base model performance, not essential for stacking) ---
            # This evaluation is on the Stage 1 task (0 vs 1)
            print(f"  评估微调后的基础模型 {cnn_name} (Stage 1) 在 Stage 1 验证集上...")
            cnn_model_for_finetuning_stage1.eval() 
            val_correct_cnn_base_stage1 = 0
            val_total_cnn_base_stage1 = 0
            all_val_labels_cnn_base_stage1 = []
            all_val_preds_cnn_base_stage1 = []

            with torch.no_grad(): 
                cnn_val_loader_tqdm_base = tqdm(val_loader_fold_stage1, desc=f'  Evaluating Base {cnn_name} (Stage 1)', leave=False)
                for batch_idx, (inputs, targets_orig) in enumerate(cnn_val_loader_tqdm_base):
                    targets_stage1_val = torch.where(targets_orig > 0, 1, 0).to(device)
                    inputs = inputs.to(device)
                    with torch.amp.autocast('cuda'):
                        outputs = cnn_model_for_finetuning_stage1(inputs)
                    _, predicted = outputs.max(1)
                    val_total_cnn_base_stage1 += targets_stage1_val.size(0)
                    val_correct_cnn_base_stage1 += predicted.eq(targets_stage1_val).sum().item()
                    all_val_labels_cnn_base_stage1.extend(targets_stage1_val.cpu().numpy())
                    all_val_preds_cnn_base_stage1.extend(predicted.cpu().numpy())

            cnn_val_accuracy_base_stage1 = 100. * val_correct_cnn_base_stage1 / val_total_cnn_base_stage1 if val_total_cnn_base_stage1 > 0 else 0
            cnn_val_macro_precision_base_stage1 = precision_score(all_val_labels_cnn_base_stage1, all_val_preds_cnn_base_stage1, average='macro', zero_division=0)
            cnn_val_macro_f1_base_stage1 = f1_score(all_val_labels_cnn_base_stage1, all_val_preds_cnn_base_stage1, average='macro', zero_division=0)
            print(f"    {fold_id_str} Base {cnn_name} Val Acc (Stage 1): {cnn_val_accuracy_base_stage1:.2f}%, Macro Precision: {cnn_val_macro_precision_base_stage1:.4f}, Macro F1: {cnn_val_macro_f1_base_stage1:.4f}")
            
            # Store the finetuned classifier
            cnn_model_for_finetuning_stage1.eval() 
            fold_cnn_finetuned_classifiers_stage1[cnn_name] = cnn_model_for_finetuning_stage1

        # --- 3. 生成 Stage 1 元特征 (来自微调后的 Stage 1 CNN 在当前折验证集上的预测概率) ---
        print(f"--- {fold_id_str}: 生成元特征 (用于 Stage 1 元模型训练) ---")
        
        current_val_meta_features_stage1_list = []   
        current_val_labels_stage1_list = [] # Stage 1 labels (0 or 1) for validation data in this fold

        # Get the actual indices for this fold's validation set
        current_val_indices_fold_stage1 = val_idx_fold_stage1 
        
        # Collect Stage 1 labels for this fold's validation set
        current_val_labels_stage1 = stage1_labels[current_val_indices_fold_stage1]

        # 对每个微调好的 Stage 1 CNN 获取其在当前折验证集上的预测概率
        for cnn_name, cnn_classifier_model in fold_cnn_finetuned_classifiers_stage1.items():
            cnn_classifier_model.eval() 
            current_val_probs_fold_list = []
            
            with torch.no_grad():
                for inputs, targets_orig in tqdm(val_loader_fold_stage1, desc=f"元特征(Stage 1 Val) from {cnn_name}", leave=False):
                    # No need to map targets here, just use inputs
                    inputs = inputs.to(device)
                    with torch.amp.autocast('cuda'):
                        outputs = cnn_classifier_model(inputs)
                        probabilities = torch.softmax(outputs, dim=1) # Get probabilities (2 classes for Stage 1 CNNs)
                    current_val_probs_fold_list.append(probabilities.cpu().numpy())
            
            current_val_meta_features_stage1_list.append(np.concatenate(current_val_probs_fold_list, axis=0))

        # Concatenate meta-features for this fold's validation set
        X_val_fold_meta_stage1 = np.concatenate(current_val_meta_features_stage1_list, axis=1)
        
        # Store for later training of the final Stage 1 meta-model
        all_oof_indices_stage1.extend(current_val_indices_fold_stage1)
        all_oof_meta_features_stage1.append(X_val_fold_meta_stage1)
        all_oof_labels_stage1.extend(current_val_labels_stage1) # Store the Stage 1 labels for this fold


    # --- Combine Stage 1 OOF data after the loop ---
    if all_oof_meta_features_stage1:
        X_oof_stage1 = np.concatenate(all_oof_meta_features_stage1, axis=0)
        y_oof_stage1 = np.array(all_oof_labels_stage1)
        oof_indices_stage1_array = np.array(all_oof_indices_stage1) # Array for easier indexing
        print(f"\nStage 1 OOF meta-features shape: {X_oof_stage1.shape}")
        print(f"Stage 1 OOF labels shape: {y_oof_stage1.shape}")
    else:
        print("\n错误: 未生成 Stage 1 OOF 元特征。跳过 Stage 1 元模型训练和后续步骤。")
        K = 0 # Set K=0 to stop further processing if Stage 1 OOF failed


In [ ]:
# --- Combine Stage 1 OOF data after the loop ---
if all_oof_meta_features_stage1:
    X_oof_stage1 = np.concatenate(all_oof_meta_features_stage1, axis=0)
    y_oof_stage1 = np.array(all_oof_labels_stage1)
    oof_indices_stage1_array = np.array(all_oof_indices_stage1) # Array for easier indexing
    print(f"\nStage 1 OOF meta-features shape: {X_oof_stage1.shape}")
    print(f"Stage 1 OOF labels shape: {y_oof_stage1.shape}")
else:
    print("\n错误: 未生成 Stage 1 OOF 元特征。跳过 Stage 1 元模型训练和后续步骤。")
    K = 0 # Set K=0 to stop further processing if Stage 1 OOF failed

# --- 清理 Stage 1 K-Fold 训练的模型占用的 GPU 内存 ---
print("\n清理 Stage 1 K-Fold 模型占用的 GPU 内存...")
del fold_cnn_finetuned_classifiers_stage1,train_loader_fold_stage1,val_loader_fold_stage1,train_dataset_fold_stage1,val_dataset_fold_stage1
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("Stage 1 K-Fold 模型内存清理完毕。")

In [ ]:
# --- 创建 Stage 2 KFold 分割器 (基于 Stage 2 数据和标签) ---
# Filter data for Stage 2 BEFORE splitting
stage2_original_df = base_full_dataset_no_transform.labels_df.iloc[stage2_indices].copy().reset_index(drop=True)
# Map original labels 1, 2 to 0, 1 for Stage 2 model training
stage2_mapped_labels = np.where(stage2_original_df['Pterygium'] == 1, 0, 1)

if K > 0 and len(stage2_mapped_labels) > 0:
    skf_stage2 = StratifiedKFold(n_splits=K, shuffle=True, random_state=base_seed)
    print(f"\n开始进行 Stage 2 ({K}-Fold Cross-Validation: 1 vs 2) on {len(stage2_mapped_labels)} samples...")
else:
    print("\nSkipping Stage 2 K-Fold Cross-Validation due to insufficient Stage 2 data or K=0.")

# --- Stage 2 K-Fold 循环 ---
if K > 0 and len(stage2_mapped_labels) > 0:
    current_fold_num = 0
    # Iterate on the indices of the stage2_original_df
    for train_idx_fold_stage2_relative, val_idx_fold_stage2_relative in skf_stage2.split(np.arange(len(stage2_original_df)), stage2_mapped_labels):
        current_fold_num += 1
        fold_id_str = f"Stage 2 - Fold {current_fold_num}/{K}"
        print(f"\n--- 开始 {fold_id_str} ---")

        # --- Map relative Stage 2 indices back to original full dataset indices ---
        train_idx_fold_stage2_original = stage2_indices[train_idx_fold_stage2_relative]
        val_idx_fold_stage2_original = stage2_indices[val_idx_fold_stage2_relative]

        # --- 1. 创建当前折的 Subset 数据集和 DataLoader (使用原始数据集和映射后的索引) ---
        train_dataset_fold_stage2 = Subset(base_full_dataset_no_transform, train_idx_fold_stage2_original)
        val_dataset_fold_stage2 = Subset(base_full_dataset_no_transform, val_idx_fold_stage2_original)

        # Apply transforms
        train_dataset_fold_stage2.dataset.transform = train_transform
        val_dataset_fold_stage2.dataset.transform = val_transform

        train_loader_fold_stage2 = DataLoader(train_dataset_fold_stage2,
                                            batch_size=64 if TRAIN_SIZE[0] <257 else 20, # 根据GPU内存调整
                                            shuffle=True,
                                            num_workers=num_workers,
                                            prefetch_factor=2 if platform.system() == "Windows" else 10,
                                            pin_memory=False)
        val_loader_fold_stage2 = DataLoader(val_dataset_fold_stage2,
                                            batch_size=64 if TRAIN_SIZE[0] <257 else 20, # 根据GPU内存调整
                                            shuffle=False,
                                            num_workers=num_workers,
                                            prefetch_factor=2 if platform.system() == "Windows" else 10,
                                            pin_memory=False)
        print(f"{fold_id_str}: Train size={len(train_dataset_fold_stage2)}, Val size={len(val_dataset_fold_stage2)}")

        # --- 2. 微调 Stage 2 CNN 基础分类器 ---
        print(f"--- {fold_id_str}: 微调 CNN 基础分类器 (Level-0 Models) ---")
        
        fold_cnn_finetuned_classifiers_stage2 = {} # 存储微调好的完整 CNN 分类器

        # CNN base models for Stage 2 predict num_classes=2 (for mapped labels 0 or 1)
        num_classes_stage2_cnn = 2 
        
        for cnn_name in CNN_FEATURE_EXTRACTORS_STAGE2:
            print(f"  微调 {cnn_name} (Stage 2)...")
            # 创建一个新的 CNN 模型实例 (带分类头), 输出层适应 2 类
            cnn_model_for_finetuning_stage2 = globals()[cnn_name](num_classes=num_classes_stage2_cnn).to(device)
            
            best_val_loss = float('inf')
            patience_counter = 0
            best_model_state_dict = None
            
            cnn_optimizer = optim.AdamW(cnn_model_for_finetuning_stage2.parameters(), 
                                        lr=cnn_micro_train_stage2_params['lr'], 
                                        weight_decay=cnn_micro_train_stage2_params['weight_decay'])
            cnn_scheduler = optim.lr_scheduler.CosineAnnealingLR(cnn_optimizer, 
                                                                T_max=cnn_micro_train_stage2_params['num_epochs'], 
                                                                eta_min=1e-6)
            cnn_criterion = nn.CrossEntropyLoss()
            scaler_cnn = torch.amp.GradScaler('cuda') 
            
            start_time_cnn_finetune = time.time()
            
            for cnn_epoch in range(cnn_micro_train_stage2_params['num_epochs']):
                cnn_model_for_finetuning_stage2.train()
                train_loss_cnn = 0
                train_correct_cnn = 0
                train_total_cnn = 0
                
                cnn_train_loader_tqdm = tqdm(train_loader_fold_stage2, desc=f'  {cnn_name} Stage 2 Epoch {cnn_epoch+1}/{cnn_micro_train_stage2_params["num_epochs"]}', leave=False)
                
                for batch_idx, (inputs, targets_orig) in enumerate(cnn_train_loader_tqdm):
                    # Map original labels (1 or 2) to Stage 2 training labels (0 or 1) for CNN training
                    targets_stage2_mapped = torch.where(targets_orig == 1, 0, 1).to(device)
                    inputs = inputs.to(device)
                    
                    cnn_optimizer.zero_grad()
                    with torch.amp.autocast('cuda'):
                        outputs = cnn_model_for_finetuning_stage2(inputs)
                        loss = cnn_criterion(outputs, targets_stage2_mapped) # Use MAPPED Stage 2 labels for loss
                    scaler_cnn.scale(loss).backward()
                    scaler_cnn.step(cnn_optimizer)
                    scaler_cnn.update()
                    train_loss_cnn += loss.item()
                    _, predicted = outputs.max(1)
                    train_total_cnn += targets_stage2_mapped.size(0)
                    train_correct_cnn += predicted.eq(targets_stage2_mapped).sum().item()
                    
                    cnn_train_loader_tqdm.set_postfix({
                        'loss': f'{loss.item():.4f}',
                        'acc': f'{100. * train_correct_cnn / train_total_cnn:.2f}%',
                        'lr': f'{cnn_optimizer.param_groups[0]["lr"]:.1e}'
                    })
                cnn_scheduler.step() # Update CNN learning rate

                # --- 基于验证集损失的早停 ---
                cnn_model_for_finetuning_stage2.eval() # Set model to evaluation mode
                val_loss_cnn = 0
                val_total_cnn = 0
                with torch.no_grad():
                    for inputs, targets_orig in val_loader_fold_stage2:
                        # Use Stage 2 mapped labels (1->0, 2->1) for validation loss
                        targets_stage2_mapped_val = torch.where(targets_orig == 1, 0, 1).to(device) 
                        inputs = inputs.to(device)
                        with torch.amp.autocast('cuda'):
                            outputs = cnn_model_for_finetuning_stage2(inputs)
                            loss = cnn_criterion(outputs, targets_stage2_mapped_val)
                        val_loss_cnn += loss.item()
                        val_total_cnn += targets_stage2_mapped_val.size(0) # Use mapped targets size

                avg_val_loss = val_loss_cnn / len(val_loader_fold_stage2) if len(val_loader_fold_stage2) > 0 else float('inf')
                print(f"    {fold_id_str} {cnn_name} Stage 2 Epoch {cnn_epoch+1}: Train Loss {train_loss_cnn/len(train_loader_fold_stage2):.4f}, Val Loss {avg_val_loss:.4f}")

                # Initialize best_val_loss and best_model_state_dict before the epoch loop
                if 'best_val_loss' not in locals() or avg_val_loss < best_val_loss:
                    best_val_loss = avg_val_loss
                    patience_counter = 0 # Reset patience
                    best_model_state_dict = cnn_model_for_finetuning_stage2.state_dict() # Save best model state
                    print(f"    {fold_id_str} {cnn_name} Val Loss improved. Saving model state.")
                else:
                    patience_counter += 1 # Increment patience if no improvement
                    print(f"    {fold_id_str} {cnn_name} Val Loss did not improve. Patience: {patience_counter}/{cnn_micro_train_stage2_params['patience']}")

                cnn_model_for_finetuning_stage2.train() # Set model back to train mode

                if patience_counter >= cnn_micro_train_stage2_params['patience']:
                    print(f"\n    {fold_id_str} {cnn_name} Stage 2 Early stopping triggered at epoch {cnn_epoch+1}.")
                    break

            # --- 训练结束后，加载验证集上表现最好的模型状态 ---
            if best_model_state_dict is not None:
                cnn_model_for_finetuning_stage2.load_state_dict(best_model_state_dict)
                print(f"    {fold_id_str} {cnn_name} Stage 2: Loaded best model state based on validation loss.")
            else:
                print(f"    {fold_id_str} {cnn_name} Stage 2: No best model state saved (maybe due to inf loss?). Using last epoch model state.")
        
            end_time_cnn_finetune = time.time()
            print(f"  微调 {cnn_name} (Stage 2) 完成，耗时: {end_time_cnn_finetune - start_time_cnn_finetune:.2f} 秒")

            # --- 评估微调后的基础 CNN 分类器在 Stage 2 验证集上的性能 ---
            # This evaluation is on the Stage 2 task (1 vs 2), using mapped labels (0 vs 1) for evaluation metrics calculation
            print(f"  评估微调后的基础模型 {cnn_name} (Stage 2) 在 Stage 2 验证集上...")
            cnn_model_for_finetuning_stage2.eval() 
            val_correct_cnn_base_stage2 = 0
            val_total_cnn_base_stage2 = 0
            all_val_labels_cnn_base_stage2_mapped = []
            all_val_preds_cnn_base_stage2_mapped = []

            with torch.no_grad(): 
                cnn_val_loader_tqdm_base = tqdm(val_loader_fold_stage2, desc=f'  Evaluating Base {cnn_name} (Stage 2)', leave=False)
                for batch_idx, (inputs, targets_orig) in enumerate(cnn_val_loader_tqdm_base):
                    targets_stage2_mapped_val = torch.where(targets_orig == 1, 0, 1).to(device)
                    inputs = inputs.to(device)
                    with torch.amp.autocast('cuda'):
                        outputs = cnn_model_for_finetuning_stage2(inputs)
                    _, predicted_mapped = outputs.max(1)
                    val_total_cnn_base_stage2 += targets_stage2_mapped_val.size(0)
                    val_correct_cnn_base_stage2 += predicted_mapped.eq(targets_stage2_mapped_val).sum().item()
                    all_val_labels_cnn_base_stage2_mapped.extend(targets_stage2_mapped_val.cpu().numpy())
                    all_val_preds_cnn_base_stage2_mapped.extend(predicted_mapped.cpu().numpy())

            cnn_val_accuracy_base_stage2 = 100. * val_correct_cnn_base_stage2 / val_total_cnn_base_stage2 if val_total_cnn_base_stage2 > 0 else 0
            cnn_val_macro_precision_base_stage2 = precision_score(all_val_labels_cnn_base_stage2_mapped, all_val_preds_cnn_base_stage2_mapped, average='macro', zero_division=0)
            cnn_val_macro_f1_base_stage2 = f1_score(all_val_labels_cnn_base_stage2_mapped, all_val_preds_cnn_base_stage2_mapped, average='macro', zero_division=0)
            print(f"    {fold_id_str} Base {cnn_name} Val Acc (Stage 2, Mapped): {cnn_val_accuracy_base_stage2:.2f}%, Macro Precision: {cnn_val_macro_precision_base_stage2:.4f}, Macro F1: {cnn_val_macro_f1_base_stage2:.4f}")
            
            # Store the finetuned classifier
            cnn_model_for_finetuning_stage2.eval() 
            fold_cnn_finetuned_classifiers_stage2[cnn_name] = cnn_model_for_finetuning_stage2


        # --- 3. 生成 Stage 2 元特征 (来自微调后的 Stage 2 CNN 在当前折 Stage 2 验证集上的预测概率) ---
        print(f"--- {fold_id_str}: 生成元特征 (用于 Stage 2 元模型训练) ---")
        
        current_val_meta_features_stage2_list = []   
        # Stage 2 labels (1 or 2) for validation data in this fold
        current_val_labels_stage2 = all_original_labels[val_idx_fold_stage2_original]

        # Get the actual indices for this fold's Stage 2 validation set
        current_val_indices_fold_stage2 = val_idx_fold_stage2_original
        
        # 对每个微调好的 Stage 2 CNN 获取其在当前折 Stage 2 验证集上的预测概率
        for cnn_name, cnn_classifier_model in fold_cnn_finetuned_classifiers_stage2.items():
            cnn_classifier_model.eval() 
            current_val_probs_fold_list = []
            
            with torch.no_grad():
                for inputs, targets_orig in tqdm(val_loader_fold_stage2, desc=f"元特征(Stage 2 Val) from {cnn_name}", leave=False):
                    # No need to map targets here, just use inputs
                    inputs = inputs.to(device)
                    with torch.amp.autocast('cuda'):
                        outputs = cnn_classifier_model(inputs)
                        probabilities = torch.softmax(outputs, dim=1) # Get probabilities (2 classes for Stage 2 CNNs)
                    current_val_probs_fold_list.append(probabilities.cpu().numpy())
            
            current_val_meta_features_stage2_list.append(np.concatenate(current_val_probs_fold_list, axis=0))

        # Concatenate meta-features for this fold's Stage 2 validation set
        X_val_fold_meta_stage2 = np.concatenate(current_val_meta_features_stage2_list, axis=1)
        
        # Store for later training of the final Stage 2 meta-model
        all_oof_indices_stage2.extend(current_val_indices_fold_stage2)
        all_oof_meta_features_stage2.append(X_val_fold_meta_stage2)
        all_oof_labels_stage2.extend(current_val_labels_stage2) # Store the ORIGINAL Stage 2 labels (1 or 2) for this fold


    # --- Combine Stage 2 OOF data after the loop ---
    if all_oof_meta_features_stage2:
        X_oof_stage2 = np.concatenate(all_oof_meta_features_stage2, axis=0)
        y_oof_stage2 = np.array(all_oof_labels_stage2) # Original labels 1 or 2
        oof_indices_stage2_array = np.array(all_oof_indices_stage2) # Array for easier indexing
        print(f"\nStage 2 OOF meta-features shape: {X_oof_stage2.shape}")
        print(f"Stage 2 OOF labels shape: {y_oof_stage2.shape}")
    else:
        print("\n错误: 未生成 Stage 2 OOF 元特征。跳过 Stage 2 元模型训练和后续步骤。")
        K = 0 # Set K=0 to stop further processing if Stage 2 OOF failed


# --- Train Meta-Models on OOF data and Evaluate Combined Performance ---
if K > 0 and all_oof_meta_features_stage1 and all_oof_meta_features_stage2:
    print("\n--- K-Fold 循环结束，开始训练元模型并评估组合性能 ---")

    # --- Train Stage 1 Meta-Model ---
    print(f"--- 训练 Stage 1 元模型 ({META_MODEL_TYPE}) on OOF data (0 vs 1) ---")
    
    # Map Stage 1 OOF labels to 0 or 1 for meta-model training
    y_oof_stage1_mapped = np.where(y_oof_stage1 > 0, 1, 0) 

    start_time_meta_stage1_train = time.time()
    meta_model_stage1_oof = None
    if META_MODEL_TYPE == 'LightGBM':
        meta_model_stage1_oof = lgb.LGBMClassifier(**lgbm_params)
        # Use split Stage 1 OOF data for early stopping? No, OOF is already the validation. Train on the full OOF.
        meta_model_stage1_oof.fit(X_oof_stage1, y_oof_stage1_mapped)
        print("Stage 1 LightGBM 元模型 (OOF) 训练完成。")
    elif META_MODEL_TYPE == 'LogisticRegression':
        meta_model_stage1_oof = LogisticRegression(**logreg_params) # LogisticRegression handles multi_class=auto
        meta_model_stage1_oof.fit(X_oof_stage1, y_oof_stage1_mapped)
        print("Stage 1 Logistic Regression 元模型 (OOF) 训练完成。")
    else:
        raise ValueError(f"Unsupported META_MODEL_TYPE for Stage 1 OOF training: {META_MODEL_TYPE}")
    end_time_meta_stage1_train = time.time()
    print(f"Stage 1 元模型训练耗时: {end_time_meta_stage1_train - start_time_meta_stage1_train:.2f} 秒")


    # --- Train Stage 2 Meta-Model ---
    print(f"\n--- 训练 Stage 2 元模型 ({META_MODEL_TYPE}) on OOF data (1 vs 2) ---")

    # Map Stage 2 OOF labels (original 1 or 2) to 0 or 1 for meta-model training
    y_oof_stage2_mapped = np.where(y_oof_stage2 == 1, 0, 1) 

    start_time_meta_stage2_train = time.time()
    meta_model_stage2_oof = None
    if META_MODEL_TYPE == 'LightGBM':
        meta_model_stage2_oof = lgb.LGBMClassifier(**lgbm_params)
        # Use split Stage 2 OOF data for early stopping? No, OOF is already the validation. Train on the full OOF.
        meta_model_stage2_oof.fit(X_oof_stage2, y_oof_stage2_mapped)
        print("Stage 2 LightGBM 元模型 (OOF) 训练完成。")
    elif META_MODEL_TYPE == 'LogisticRegression':
        meta_model_stage2_oof = LogisticRegression(**logreg_params) # LogisticRegression handles multi_class=auto
        meta_model_stage2_oof.fit(X_oof_stage2, y_oof_stage2_mapped)
        print("Stage 2 Logistic Regression 元模型 (OOF) 训练完成。")
    else:
        raise ValueError(f"Unsupported META_MODEL_TYPE for Stage 2 OOF training: {META_MODEL_TYPE}")
    end_time_meta_stage2_train = time.time()
    print(f"Stage 2 元模型训练耗时: {end_time_meta_stage2_train - start_time_meta_stage2_train:.2f} 秒")


    # --- Combine Predictions and Evaluate Overall 3-Class Metrics on OOF data ---
    print("\n--- 评估 Stacking Ensemble (两阶段) 在 OOF 数据上的组合性能 (3 类) ---")

    # Predict Stage 1 outcome for all OOF samples
    y_pred_stage1_oof_mapped = meta_model_stage1_oof.predict(X_oof_stage1) # 0 or 1

    # Initialize final predictions array for all OOF samples
    y_combined_pred_oof = np.zeros_like(y_oof_stage1_mapped)

    # Samples predicted as Normal (0) by Stage 1
    normal_predicted_indices_in_oof = oof_indices_stage1_array[y_pred_stage1_oof_mapped == 0]
    y_combined_pred_oof[y_pred_stage1_oof_mapped == 0] = 0 # Final prediction is 0

    # Samples predicted as Diseased (>0) by Stage 1
    diseased_predicted_indices_in_oof = oof_indices_stage1_array[y_pred_stage1_oof_mapped == 1]

    # For the samples predicted as diseased by Stage 1, we need Stage 2 prediction.
    # We need to find which of these indices are present in the Stage 2 OOF data.
    # This requires careful indexing. We can map the indices.
    
    # Create a mapping from original index to its position in Stage 2 OOF data
    stage2_oof_index_map = {original_idx: i for i, original_idx in enumerate(oof_indices_stage2_array)}

    # Find indices in X_oof_stage2 that correspond to samples predicted as diseased by Stage 1
    # These are the samples we need to predict using the Stage 2 meta-model
    indices_for_stage2_prediction_in_oof_stage2_data = [
        stage2_oof_index_map[original_idx]
        for original_idx in diseased_predicted_indices_in_oof
        if original_idx in stage2_oof_index_map # Ensure the index exists in Stage 2 OOF (should always be true if Stage 1 predicted >0)
    ]
    
    # Get the subset of Stage 2 OOF meta-features for these samples
    X_oof_stage2_subset_for_prediction = X_oof_stage2[indices_for_stage2_prediction_in_oof_stage2_data]

    # Predict using Stage 2 meta-model for these samples
    if len(X_oof_stage2_subset_for_prediction) > 0:
        y_pred_stage2_oof_mapped_subset = meta_model_stage2_oof.predict(X_oof_stage2_subset_for_prediction) # 0 or 1
        # Map Stage 2 predictions back to original labels (0->1, 1->2)
        y_pred_stage2_oof_original_subset = np.where(y_pred_stage2_oof_mapped_subset == 0, 1, 2) # 1 or 2

        # Place these predictions into the correct locations in the combined prediction array
        # We need the indices *in the y_combined_pred_oof array* that correspond to these Stage 2 predictions
        # These are the indices in y_pred_stage1_oof_mapped == 1
        indices_to_update_in_combined_oof = np.where(y_pred_stage1_oof_mapped == 1)[0]

        # Ensure the lengths match - they should if the logic is correct
        if len(indices_to_update_in_combined_oof) == len(y_pred_stage2_oof_original_subset):
            y_combined_pred_oof[indices_to_update_in_combined_oof] = y_pred_stage2_oof_original_subset
        else:
            print("错误: Stage 1 预测为有病的数量与 Stage 2 对应 OOF 数据数量不匹配。组合预测可能出错。")
            # Fallback: Could set these to a default (e.g., 1), or raise error. Let's print warning and use Stage 1 prediction as fallback (will result in 1 for all)
            y_combined_pred_oof[indices_to_update_in_combined_oof] = 1 # Fallback to 1 if Stage 2 fails

    else:
        print("Stage 1 未预测任何样本为有病 (>0)。Stage 2 模型未用于 OOF 预测。")


    # Get the true original labels corresponding to the OOF data indices
    # This requires mapping back using oof_indices_stage1_array
    y_true_oof_original = all_original_labels[oof_indices_stage1_array]

    # Calculate metrics using the combined predictions and original true labels
    oof_accuracy = accuracy_score(y_true_oof_original, y_combined_pred_oof)
    oof_macro_precision = precision_score(y_true_oof_original, y_combined_pred_oof, average='macro', zero_division=0)
    oof_macro_f1 = f1_score(y_true_oof_original, y_combined_pred_oof, average='macro', zero_division=0)

    print("\n--- Stacked Ensemble (两阶段) K-Fold OOF 评估结果 (3 类) ---")
    print(f"OOF 准确率: {oof_accuracy:.4f}")
    print(f"OOF Macro Precision: {oof_macro_precision:.4f}")
    print(f"OOF Macro F1: {oof_macro_f1:.4f}")

    # Optional: Confusion Matrix on OOF
    conf_matrix_oof = confusion_matrix(y_true_oof_original, y_combined_pred_oof)
    print("\nOOF 混淆矩阵:")
    print(conf_matrix_oof)

else:
    print("\nSkipping K-Fold evaluation due to errors in previous steps.")


# --- Clean up temporary files if any were used ---
# (No temporary files like excel were used in this revised K-fold)
# if 'temp_dir_for_folds' in locals() and os.path.exists(temp_dir_for_folds):
#     print(f"清理临时文件夹: {temp_dir_for_folds}")
#     try:
#         shutil.rmtree(temp_dir_for_folds)
#     except Exception as e:
#         print(f"警告: 清理临时文件夹失败: {e}")

In [ ]:
# --- Combine Stage 2 OOF data after the loop ---
if all_oof_meta_features_stage2:
    X_oof_stage2 = np.concatenate(all_oof_meta_features_stage2, axis=0)
    y_oof_stage2 = np.array(all_oof_labels_stage2) # Original labels 1 or 2
    oof_indices_stage2_array = np.array(all_oof_indices_stage2) # Array for easier indexing
    print(f"\nStage 2 OOF meta-features shape: {X_oof_stage2.shape}")
    print(f"Stage 2 OOF labels shape: {y_oof_stage2.shape}")
else:
    print("\n错误: 未生成 Stage 2 OOF 元特征。跳过 Stage 2 元模型训练和后续步骤。")
    K = 0 # Set K=0 to stop further processing if Stage 2 OOF failed

# --- 清理 Stage 2 K-Fold 训练的模型占用的 GPU 内存 ---
print("\n清理 Stage 2 K-Fold 模型占用的 GPU 内存...")
del fold_cnn_finetuned_classifiers_stage2,train_loader_fold_stage2,val_loader_fold_stage2,train_dataset_fold_stage2,val_dataset_fold_stage2
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("Stage 2 K-Fold 模型内存清理完毕。")

# --- Train Meta-Models on OOF data and Evaluate Combined Performance ---
if K > 0 and all_oof_meta_features_stage1 and all_oof_meta_features_stage2:
    print("\n--- K-Fold 循环结束，开始训练元模型并评估组合性能 ---")

# 训练最终用于提交的模型 (两阶段)
使用整个 Stage 1 和 Stage 2 训练集训练 CNN 基础分类器，然后使用它们的预测概率作为元特征，训练最终的元模型。

In [ ]:
print("\n--- 开始训练最终提交的两阶段 Stacking 集成模型 ---")

# Use the full_labels_df and base_full_dataset_no_transform defined earlier

# --- Prepare data for Final Stage 1 Training ---
# Stage 1 uses the entire dataset with mapped labels (0 or 1)
final_stage1_labels_mapped = np.where(full_labels_df['Pterygium'] > 0, 1, 0)
# Create a temporary DataFrame with mapped labels for DataLoader (or just use the full_labels_df and map inside)
# Let's map inside the getitem for simplicity if we use Subset of the original dataset
# Alternatively, create a new dataset class or modify PterygiumDataset to accept a label column name

# Option 1: Modify PterygiumDataset slightly to take label column name (More flexible)
class PterygiumDatasetWithLabelCol(Dataset):
    def __init__(self, label_df, image_dir, label_col='Pterygium', transform=None):
        self.labels_df = label_df.reset_index(drop=True) # Ensure continuous index
        self.image_dir = image_dir
        self.transform = transform
        self.label_col = label_col
        
        self.image_paths = []
        for index, row in self.labels_df.iterrows():
            image_name = row['Image']
            image_folder = f"{int(image_name):04d}"
            potential_path_subfolder = os.path.join(self.image_dir, image_folder, f"{image_folder}.png")
            potential_path_direct = os.path.join(self.image_dir, f"{image_folder}.png")

            if os.path.exists(potential_path_subfolder):
                self.image_paths.append(potential_path_subfolder)
            elif os.path.exists(potential_path_direct):
                self.image_paths.append(potential_path_direct)
            else:
                print(f"错误: Final training image file not found {image_folder}.png in {self.image_dir}")
                self.image_paths.append(None)
        
        # Filter out samples where image file was not found
        valid_indices = [i for i, path in enumerate(self.image_paths) if path is not None]
        if len(valid_indices) < len(self.labels_df):
            print(f"警告: 发现 {len(self.labels_df) - len(valid_indices)} 图像文件在 {image_dir} 中缺失，这些样本将被跳过。")
        self.labels_df = self.labels_df.iloc[valid_indices].reset_index(drop=True)
        self.image_paths = [self.image_paths[i] for i in valid_indices]
        
    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels_df.iloc[idx][self.label_col]

        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label


# Add Stage 1 Mapped Label column to the full DataFrame
full_labels_df_with_stage1 = full_labels_df.copy()
full_labels_df_with_stage1['Pterygium_Stage1'] = np.where(full_labels_df_with_stage1['Pterygium'] > 0, 1, 0)

final_train_dataset_stage1 = PterygiumDatasetWithLabelCol(
    label_df=full_labels_df_with_stage1, 
    image_dir=image_dir, 
    label_col='Pterygium_Stage1', # Use Stage 1 mapped label
    transform=train_transform # Use training transform
)
final_train_loader_stage1 = DataLoader(
    final_train_dataset_stage1,
    batch_size=64 if TRAIN_SIZE[0] <257 else 30,
    shuffle=True,
    num_workers=num_workers,
    prefetch_factor=2 if platform.system() == "Windows" else 10,
    pin_memory=False
)

# Prepare data for Final Stage 2 Training
# Stage 2 uses the subset of data where original label is 1 or 2
final_stage2_df_original = full_labels_df[full_labels_df['Pterygium'] > 0].copy()
# Add Stage 2 Mapped Label column (1->0, 2->1) for training
final_stage2_df_original['Pterygium_Stage2_Mapped'] = np.where(final_stage2_df_original['Pterygium'] == 1, 0, 1)


final_train_dataset_stage2 = PterygiumDatasetWithLabelCol(
    label_df=final_stage2_df_original, 
    image_dir=image_dir, 
    label_col='Pterygium_Stage2_Mapped', # Use Stage 2 mapped label for training
    transform=train_transform # Use training transform
)
final_train_loader_stage2 = DataLoader(
    final_train_dataset_stage2,
    batch_size=64 if TRAIN_SIZE[0] <257 else 20,
    shuffle=True,
    num_workers=num_workers,
    prefetch_factor=2 if platform.system() == "Windows" else 10,
    pin_memory=False
)

# Data loader to generate meta-features for the *entire* training set (using val_transform)
# Needed for training the final meta-models
final_meta_train_dataset_full = PterygiumDatasetWithLabelCol(
    label_df=full_labels_df, # Use original labels, but won't use them in meta-feature extraction
    image_dir=image_dir,
    label_col='Pterygium', # This column is not used in the loop below, just for dataset structure
    transform=val_transform # Use validation transform for consistent meta-features
)
final_meta_train_loader_full = DataLoader(
    final_meta_train_dataset_full,
    batch_size=64,
    shuffle=False, # Do not shuffle for meta-feature extraction
    num_workers=num_workers,
    prefetch_factor=2 if platform.system() == "Windows" else 10,
    pin_memory=False
)


# Data loader to generate meta-features for the Stage 2 training subset (using val_transform)
final_meta_train_dataset_stage2_subset = PterygiumDatasetWithLabelCol(
    label_df=final_stage2_df_original, # Use the filtered Stage 2 DataFrame
    image_dir=image_dir,
    label_col='Pterygium', # Original label column
    transform=val_transform # Use validation transform
)
final_meta_train_loader_stage2_subset = DataLoader(
    final_meta_train_dataset_stage2_subset,
    batch_size=64,
    shuffle=False, # Do not shuffle
    num_workers=num_workers,
    prefetch_factor=2 if platform.system() == "Windows" else 10,
    pin_memory=False
)


# --- 1. 训练最终 Stage 1 CNN 基础分类器 (在整个训练集上, 0 vs >0) ---
print("--- 步骤 1: 训练最终 Stage 1 CNN 基础分类器 (在整个训练集上, 0 vs >0) ---")

final_cnn_classifiers_stage1 = {} # 存储最终训练好的完整 CNN 分类器

num_classes_stage1_cnn = 2 # Stage 1 CNNs predict 2 classes (0 or 1)

for cnn_name in CNN_FEATURE_EXTRACTORS_STAGE1:
    print(f"  训练基础模型: {cnn_name} (Stage 1) 在完整数据集上...")
    cnn_model_for_final_training_stage1 = globals()[cnn_name](num_classes=num_classes_stage1_cnn).to(device)

    final_train_epochs = cnn_micro_train_stage1_params.get('num_epochs_final', cnn_micro_train_stage1_params['num_epochs']) 
    
    cnn_optimizer_final = optim.AdamW(cnn_model_for_final_training_stage1.parameters(), 
                                        lr=cnn_micro_train_stage1_params['lr'], 
                                        weight_decay=cnn_micro_train_stage1_params['weight_decay'])
    cnn_scheduler_final = optim.lr_scheduler.CosineAnnealingLR(cnn_optimizer_final, 
                                                                T_max=final_train_epochs, 
                                                                eta_min=1e-6)
    cnn_criterion_final = nn.CrossEntropyLoss()
    scaler_cnn_final = torch.amp.GradScaler('cuda') 
    
    start_time_cnn_final_train = time.time()
    
    for cnn_epoch in range(final_train_epochs):
        cnn_model_for_final_training_stage1.train()
        train_loss_cnn_final = 0
        train_correct_cnn_final = 0
        train_total_cnn_final = 0
        
        cnn_train_loader_final_tqdm = tqdm(final_train_loader_stage1, desc=f'  {cnn_name} Stage 1 Final Epoch {cnn_epoch+1}/{final_train_epochs}', leave=False)
        
        for batch_idx, (inputs, targets_stage1_mapped) in enumerate(cnn_train_loader_final_tqdm):
            inputs, targets_stage1_mapped = inputs.to(device), targets_stage1_mapped.to(device)
            cnn_optimizer_final.zero_grad()
            with torch.amp.autocast('cuda'):
                outputs = cnn_model_for_final_training_stage1(inputs)
                loss = cnn_criterion_final(outputs, targets_stage1_mapped) # Use Stage 1 mapped labels
            scaler_cnn_final.scale(loss).backward()
            scaler_cnn_final.step(cnn_optimizer_final)
            scaler_cnn_final.update()
            
            train_loss_cnn_final += loss.item()
            _, predicted = outputs.max(1)
            train_total_cnn_final += targets_stage1_mapped.size(0)
            train_correct_cnn_final += predicted.eq(targets_stage1_mapped).sum().item()
            
            cnn_train_loader_final_tqdm.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100. * train_correct_cnn_final / train_total_cnn_final:.2f}%' if train_total_cnn_final > 0 else '0.00%',
                'lr': f'{cnn_optimizer_final.param_groups[0]["lr"]:.1e}'
            })
        cnn_scheduler_final.step()
    
    end_time_cnn_final_train = time.time()
    print(f"  训练 Stage 1 基础模型 {cnn_name} 完成，耗时: {end_time_cnn_final_train - start_time_cnn_final_train:.2f} 秒")

    cnn_model_for_final_training_stage1.eval() 
    final_cnn_classifiers_stage1[cnn_name] = cnn_model_for_final_training_stage1
    
    # Save Stage 1 base classifier state dict
    base_classifier_save_path_stage1 = f'./final_{cnn_name}_stage1_base_classifier.pth'
    save_cnn_state_dict(cnn_model_for_final_training_stage1, base_classifier_save_path_stage1)
    print(f"  Stage 1 基础分类器 {cnn_name} 已保存到 {base_classifier_save_path_stage1}")

In [ ]:
print("\n--- 清理 Stage 1 最终训练模型占用的 GPU 内存 ---")
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("Stage 1 最终训练模型内存清理完毕。")

In [ ]:
# --- 2. 训练最终 Stage 2 CNN 基础分类器 (在 Stage 2 子集上, 1 vs 2) ---
print("\n--- 步骤 2: 训练最终 Stage 2 CNN 基础分类器 (在 Stage 2 子集上, 1 vs 2) ---")

final_cnn_classifiers_stage2 = {} # 存储最终训练好的完整 CNN 分类器

num_classes_stage2_cnn = 2 # Stage 2 CNNs predict 2 classes (for mapped 0 or 1)

for cnn_name in CNN_FEATURE_EXTRACTORS_STAGE2:
    print(f"  训练基础模型: {cnn_name} (Stage 2) 在 Stage 2 数据集上...")
    cnn_model_for_final_training_stage2 = globals()[cnn_name](num_classes=num_classes_stage2_cnn).to(device)

    # Use same training params as Stage 1 for consistency
    final_train_epochs = cnn_micro_train_stage2_params.get('num_epochs_final', cnn_micro_train_stage2_params['num_epochs']) 

    cnn_optimizer_final = optim.AdamW(cnn_model_for_final_training_stage2.parameters(), 
                                        lr=cnn_micro_train_stage2_params['lr'], 
                                        weight_decay=cnn_micro_train_stage2_params['weight_decay'])
    cnn_scheduler_final = optim.lr_scheduler.CosineAnnealingLR(cnn_optimizer_final, 
                                                                T_max=final_train_epochs, 
                                                                eta_min=1e-6)
    cnn_criterion_final = nn.CrossEntropyLoss()
    scaler_cnn_final = torch.amp.GradScaler('cuda') 
    
    start_time_cnn_final_train = time.time()
    
    for cnn_epoch in range(final_train_epochs):
        cnn_model_for_final_training_stage2.train()
        train_loss_cnn_final = 0
        train_correct_cnn_final = 0
        train_total_cnn_final = 0
        
        cnn_train_loader_final_tqdm = tqdm(final_train_loader_stage2, desc=f'  {cnn_name} Stage 2 Final Epoch {cnn_epoch+1}/{final_train_epochs}', leave=False)
        
        for batch_idx, (inputs, targets_stage2_mapped) in enumerate(cnn_train_loader_final_tqdm):
            inputs, targets_stage2_mapped = inputs.to(device), targets_stage2_mapped.to(device)
            cnn_optimizer_final.zero_grad()
            with torch.amp.autocast('cuda'):
                outputs = cnn_model_for_final_training_stage2(inputs)
                loss = cnn_criterion_final(outputs, targets_stage2_mapped) # Use Stage 2 mapped labels (0 or 1)
            scaler_cnn_final.scale(loss).backward()
            scaler_cnn_final.step(cnn_optimizer_final)
            scaler_cnn_final.update()
            
            train_loss_cnn_final += loss.item()
            _, predicted = outputs.max(1)
            train_total_cnn_final += targets_stage2_mapped.size(0)
            train_correct_cnn_final += predicted.eq(targets_stage2_mapped).sum().item()
            
            cnn_train_loader_final_tqdm.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100. * train_correct_cnn_final / train_total_cnn_final:.2f}%' if train_total_cnn_final > 0 else '0.00%',
                'lr': f'{cnn_optimizer_final.param_groups[0]["lr"]:.1e}'
            })
        cnn_scheduler_final.step()
    
    end_time_cnn_final_train = time.time()
    print(f"  训练 Stage 2 基础模型 {cnn_name} 完成，耗时: {end_time_cnn_final_train - start_time_cnn_final_train:.2f} 秒")

    cnn_model_for_final_training_stage2.eval() 
    final_cnn_classifiers_stage2[cnn_name] = cnn_model_for_final_training_stage2
    
    # Save Stage 2 base classifier state dict
    base_classifier_save_path_stage2 = f'./final_{cnn_name}_stage2_base_classifier.pth'
    save_cnn_state_dict(cnn_model_for_final_training_stage2, base_classifier_save_path_stage2)
    print(f"  Stage 2 基础分类器 {cnn_name} 已保存到 {base_classifier_save_path_stage2}")

In [ ]:
print("\n--- 清理 Stage 2 最终训练模型占用的 GPU 内存 ---")
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("Stage 2 最终训练模型内存清理完毕。")

In [ ]:
# --- 3. 为 Stage 1 元模型提取最终训练特征 (从整个训练集，使用最终 Stage 1 CNN 分类器) ---
print("\n--- 步骤 3: 为 Stage 1 元模型提取整个训练集的预测概率 (元特征) ---")

final_train_meta_features_stage1_list = []

if not final_cnn_classifiers_stage1:
    print("错误：没有训练好的 Stage 1 基础CNN分类器。无法提取元特征。")
    # sys.exit("无法继续元特征提取。")
else:
    # 对每个训练好的 Stage 1 CNN 分类器提取预测概率
    for cnn_name, cnn_classifier_model in final_cnn_classifiers_stage1.items():
        print(f"  使用 Stage 1 基础模型 {cnn_name} 提取完整训练集预测概率...")
        cnn_classifier_model.eval()
        current_final_train_probs_list = []
        
        with torch.no_grad():
            # Use final_meta_train_loader_full (entire training set, val_transform)
            for inputs, _ in tqdm(final_meta_train_loader_full, desc=f"元特征(Stage 1 Final Train) from {cnn_name}", leave=False):
                inputs = inputs.to(device)
                #with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                outputs = cnn_classifier_model(inputs)
                probabilities = torch.softmax(outputs, dim=1) # 2 classes
                current_final_train_probs_list.append(probabilities.cpu().numpy())
        
        final_train_meta_features_stage1_list.append(np.concatenate(current_final_train_probs_list, axis=0))

    if not final_train_meta_features_stage1_list:
        print("错误：未能生成 Stage 1 元特征。")
        # sys.exit("Stage 1 元特征生成失败。")
    else:
        X_final_train_meta_stage1 = np.concatenate(final_train_meta_features_stage1_list, axis=1)
        # The labels for Stage 1 meta-model training are the mapped labels (0 or 1) for the full training set
        y_final_train_stage1_mapped = final_train_dataset_stage1.labels_df['Pterygium_Stage1'].values # Get labels from the dataset df
        
        print(f"拼接后 Stage 1 最终训练集元特征形状: {X_final_train_meta_stage1.shape}")
        print(f"Stage 1 最终训练集元标签形状: {y_final_train_stage1_mapped.shape}")


# --- 4. 为 Stage 2 元模型提取最终训练特征 (从 Stage 2 子集，使用最终 Stage 2 CNN 分类器) ---
print("\n--- 步骤 4: 为 Stage 2 元模型提取 Stage 2 训练子集的预测概率 (元特征) ---")

final_train_meta_features_stage2_list = []

if not final_cnn_classifiers_stage2:
    print("错误：没有训练好的 Stage 2 基础CNN分类器。无法提取元特征。")
    # sys.exit("无法继续元特征提取。")
else:
    # 对每个训练好的 Stage 2 CNN 分类器提取预测概率
    for cnn_name, cnn_classifier_model in final_cnn_classifiers_stage2.items():
        print(f"  使用 Stage 2 基础模型 {cnn_name} 提取 Stage 2 训练子集预测概率...")
        cnn_classifier_model.eval()
        current_final_train_probs_list = []
        
        with torch.no_grad():
            # Use final_meta_train_loader_stage2_subset (Stage 2 training data, val_transform)
            for inputs, _ in tqdm(final_meta_train_loader_stage2_subset, desc=f"元特征(Stage 2 Final Train) from {cnn_name}", leave=False):
                inputs = inputs.to(device)
                #with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                outputs = cnn_classifier_model(inputs)
                probabilities = torch.softmax(outputs, dim=1) # 2 classes
                current_final_train_probs_list.append(probabilities.cpu().numpy())
        
        final_train_meta_features_stage2_list.append(np.concatenate(current_final_train_probs_list, axis=0))

    if not final_train_meta_features_stage2_list:
        print("错误：未能生成 Stage 2 元特征。")
        # sys.exit("Stage 2 元特征生成失败。")
    else:
        X_final_train_meta_stage2 = np.concatenate(final_train_meta_features_stage2_list, axis=1)
        # The labels for Stage 2 meta-model training are the mapped labels (0 or 1) for the Stage 2 subset
        y_final_train_stage2_mapped = final_train_dataset_stage2.labels_df['Pterygium_Stage2_Mapped'].values # Get labels from the dataset df
        # Store original Stage 2 labels for potential later checks if needed
        y_final_train_stage2_original = final_train_dataset_stage2.labels_df['Pterygium'].values

        print(f"拼接后 Stage 2 最终训练集元特征形状: {X_final_train_meta_stage2.shape}")
        print(f"Stage 2 最终训练集元标签形状: {y_final_train_stage2_mapped.shape}")


# --- 5. 训练最终的元模型 (Stage 1 和 Stage 2) ---
print(f"\n--- 步骤 5: 训练最终 Stage 1 元模型 ({META_MODEL_TYPE}) ---")

final_meta_model_stage1 = None
if 'X_final_train_meta_stage1' in locals() and 'y_final_train_stage1_mapped' in locals():
    start_time_final_meta_train = time.time()
    if META_MODEL_TYPE == 'LightGBM':
        final_meta_model_stage1 = lgb.LGBMClassifier(**lgbm_params)
        final_meta_model_stage1.fit(X_final_train_meta_stage1, y_final_train_stage1_mapped)
        print("最终 Stage 1 LightGBM 元模型训练完成。")
    elif META_MODEL_TYPE == 'LogisticRegression':
        final_meta_model_stage1 = LogisticRegression(**logreg_params)
        final_meta_model_stage1.fit(X_final_train_meta_stage1, y_final_train_stage1_mapped)
        print("最终 Stage 1 Logistic Regression 元模型训练完成。")
    else:
        raise ValueError(f"Unsupported META_MODEL_TYPE for final Stage 1 training: {META_MODEL_TYPE}")
    end_time_final_meta_train = time.time()
    print(f"最终 Stage 1 元模型训练耗时: {end_time_final_meta_train - start_time_final_meta_train:.2f} 秒")
else:
    print("错误: Stage 1 最终训练元特征或标签不存在。跳过 Stage 1 最终元模型训练。")


print(f"\n--- 步骤 6: 训练最终 Stage 2 元模型 ({META_MODEL_TYPE}) ---")

final_meta_model_stage2 = None
if 'X_final_train_meta_stage2' in locals() and 'y_final_train_stage2_mapped' in locals():
    start_time_final_meta_train = time.time()
    if META_MODEL_TYPE == 'LightGBM':
        final_meta_model_stage2 = lgb.LGBMClassifier(**lgbm_params)
        final_meta_model_stage2.fit(X_final_train_meta_stage2, y_final_train_stage2_mapped)
        print("最终 Stage 2 LightGBM 元模型训练完成。")
    elif META_MODEL_TYPE == 'LogisticRegression':
        final_meta_model_stage2 = LogisticRegression(**logreg_params)
        final_meta_model_stage2.fit(X_final_train_meta_stage2, y_final_train_stage2_mapped)
        print("最终 Stage 2 Logistic Regression 元模型训练完成。")
    else:
        raise ValueError(f"Unsupported META_MODEL_TYPE for final Stage 2 training: {META_MODEL_TYPE}")
    end_time_final_meta_train = time.time()
    print(f"最终 Stage 2 元模型训练耗时: {end_time_final_meta_train - start_time_final_meta_train:.2f} 秒")
else:
    print("错误: Stage 2 最终训练元特征或标签不存在。跳过 Stage 2 最终元模型训练。")


# --- 7. 保存最终的模型 ---
print("\n--- 步骤 7: 保存最终模型 ---")

if final_meta_model_stage1:
    final_meta_model_save_path_stage1 = f"./final_stacked_meta_classifier_stage1_{META_MODEL_TYPE.lower().replace(' ', '_')}.joblib" 
    save_boosting_model(final_meta_model_stage1, final_meta_model_save_path_stage1)
else:
    print(" Stage 1 元模型未成功训练，未保存。")

if final_meta_model_stage2:
    final_meta_model_save_path_stage2 = f"./final_stacked_meta_classifier_stage2_{META_MODEL_TYPE.lower().replace(' ', '_')}.joblib" 
    save_boosting_model(final_meta_model_stage2, final_meta_model_save_path_stage2)
else:
    print(" Stage 2 元模型未成功训练，未保存。")


print("\n--- 最终提交的两阶段 Stacking 集成模型训练完成 ---")
print(f"基础CNN分类器已保存 (例如: ./final_{CNN_FEATURE_EXTRACTORS_STAGE1[0]}_stage1_base_classifier.pth)")
if final_meta_model_stage1:
    print(f"最终 Stage 1 元模型已保存到: {final_meta_model_save_path_stage1}")
if final_meta_model_stage2:
    print(f"最终 Stage 2 元模型已保存到: {final_meta_model_save_path_stage2}")

# 最终 Stacking 集成模型预测 (两阶段)
加载训练好的 Stage 1 和 Stage 2 模型，并使用两阶段逻辑对新图像进行预测。

In [ ]:
print("\n--- 开始加载最终模型进行预测 ---")

# Load Stage 1 Meta Model
final_meta_model_load_path_stage1 = f"./final_stacked_meta_classifier_stage1_{META_MODEL_TYPE.lower().replace(' ', '_')}.joblib"
final_meta_model_stage1_loaded = load_boosting_model(final_meta_model_load_path_stage1)
if not final_meta_model_stage1_loaded:
    print("错误: Stage 1 最终元模型加载失败。预测将无法进行。")
    sys.exit("模型加载失败，无法进行预测。")

# Load Stage 2 Meta Model
final_meta_model_load_path_stage2 = f"./final_stacked_meta_classifier_stage2_{META_MODEL_TYPE.lower().replace(' ', '_')}.joblib"
final_meta_model_stage2_loaded = load_boosting_model(final_meta_model_load_path_stage2)
if not final_meta_model_stage2_loaded:
    print("错误: Stage 2 最终元模型加载失败。预测将无法进行。")
    sys.exit("模型加载失败，无法进行预测。")


# Load Final Stage 1 CNN Base Classifiers
final_cnn_classifiers_stage1_loaded = {}
print("加载 Stage 1 CNN 基础分类器...")
all_base_stage1_loaded = True
num_classes_stage1_cnn = 2 # Stage 1 CNNs were trained for 2 classes
for cnn_name in CNN_FEATURE_EXTRACTORS_STAGE1:
    print(f"  加载 {cnn_name} (Stage 1) 基础分类器...")
    base_classifier_model_instance = globals()[cnn_name](num_classes=num_classes_stage1_cnn) 
    final_cnn_state_dict_path = f'./final_{cnn_name}_stage1_base_classifier.pth'
    loaded_model = load_cnn_state_dict(base_classifier_model_instance, final_cnn_state_dict_path, device)
    if loaded_model:
        loaded_model.eval() # Ensure eval mode
        final_cnn_classifiers_stage1_loaded[cnn_name] = loaded_model
    else:
        all_base_stage1_loaded = False
        final_cnn_classifiers_stage1_loaded[cnn_name] = None # Store None to indicate failure

if not all_base_stage1_loaded:
    print("警告: 一个或多个 Stage 1 基础CNN分类器未能加载。预测可能不准确或失败。")
    # Decide if this is a fatal error or just a warning
    # sys.exit("Stage 1 基础模型加载不完整，无法预测。")


# Load Final Stage 2 CNN Base Classifiers
final_cnn_classifiers_stage2_loaded = {}
print("\n加载 Stage 2 CNN 基础分类器...")
all_base_stage2_loaded = True
num_classes_stage2_cnn = 2 # Stage 2 CNNs were trained for 2 classes
for cnn_name in CNN_FEATURE_EXTRACTORS_STAGE2:
    print(f"  加载 {cnn_name} (Stage 2) 基础分类器...")
    base_classifier_model_instance = globals()[cnn_name](num_classes=num_classes_stage2_cnn) 
    final_cnn_state_dict_path = f'./final_{cnn_name}_stage2_base_classifier.pth'
    loaded_model = load_cnn_state_dict(base_classifier_model_instance, final_cnn_state_dict_path, device)
    if loaded_model:
        loaded_model.eval() # Ensure eval mode
        final_cnn_classifiers_stage2_loaded[cnn_name] = loaded_model
    else:
        all_base_stage2_loaded = False
        final_cnn_classifiers_stage2_loaded[cnn_name] = None # Store None to indicate failure

if not all_base_stage2_loaded:
    print("警告: 一个或多个 Stage 2 基础CNN分类器未能加载。Stage 2 预测可能不准确或失败。")
    # Decide if this is a fatal error
    # sys.exit("Stage 2 基础模型加载不完整，无法预测。")


def predict_single_image_two_stage(
    cnn_base_classifiers_stage1_dict, 
    meta_model_stage1_instance,
    cnn_base_classifiers_stage2_dict,
    meta_model_stage2_instance,
    image_path, 
    prediction_transform, 
    device_to_use):
    """
    使用两阶段 Stacking Ensemble 模型对单张图像进行预测。

    参数:
        cnn_base_classifiers_stage1_dict (dict): 已加载的 Stage 1 基础CNN分类器字典。
        meta_model_stage1_instance: 已加载的 Stage 1 元模型。
        cnn_base_classifiers_stage2_dict (dict): 已加载的 Stage 2 基础CNN分类器字典。
        meta_model_stage2_instance: 已加载的 Stage 2 元模型。
        image_path (str): 待预测图像的路径。
        prediction_transform: 应用于预测图像的 torchvision transform。
        device_to_use: 'cuda' 或 'cpu'。

    返回:
        int or None: 预测的类别标签 (0, 1, or 2)，如果出错则为 None。
    """
    # 1. 检查模型是否都已加载且有效
    if not meta_model_stage1_instance or not meta_model_stage2_instance:
        print("错误: Stage 1 或 Stage 2 元模型未加载。")
        return None
    if not cnn_base_classifiers_stage1_dict or not all(cnn_base_classifiers_stage1_dict.values()):
        print("错误: Stage 1 基础模型未完全加载。")
        return None
    if not cnn_base_classifiers_stage2_dict or not all(cnn_base_classifiers_stage2_dict.values()):
        print("错误: Stage 2 基础模型未完全加载。")
        return None


    # 2. 加载和预处理图像
    try:
        image = Image.open(image_path).convert("RGB")
        image_tensor = prediction_transform(image).unsqueeze(0).to(device_to_use)
    except FileNotFoundError:
        print(f"错误: 预测图像文件未找到 {image_path}")
        return None 
    except Exception as e:
        print(f"错误处理预测图像 {image_path}: {e}")
        return None

    # 3. Stage 1 预测: 生成 Stage 1 元特征并使用 Stage 1 元模型预测
    meta_features_stage1_list = []
    with torch.no_grad():
        for cnn_name, cnn_classifier_model in cnn_base_classifiers_stage1_dict.items():
            # We already checked for None above, but double check for safety
            if cnn_classifier_model is None: continue
            #with torch.amp.autocast('cuda' if device_to_use == 'cuda' else 'cpu', enabled=torch.cuda.is_available()):
            outputs = cnn_classifier_model(image_tensor)
            probabilities = torch.softmax(outputs, dim=1) # 2 classes for Stage 1
            meta_features_stage1_list.append(probabilities.cpu().numpy())

    if not meta_features_stage1_list or len(meta_features_stage1_list) != len(cnn_base_classifiers_stage1_dict):
        print("错误: Stage 1 基础模型元特征生成失败。")
        return None
        
    image_meta_features_stage1 = np.concatenate(meta_features_stage1_list, axis=1)

    # Predict Stage 1 outcome (0: Normal, 1: Diseased)
    try:
        # meta_model_stage1_instance predicts 0 or 1 (mapped Stage 1 labels)
        predicted_stage1_mapped = meta_model_stage1_instance.predict(image_meta_features_stage1)[0]
    except Exception as e:
        print(f"Stage 1 元模型预测时出错: {e}")
        return None

    # 4. Determine final prediction based on Stage 1 outcome
    if predicted_stage1_mapped == 0:
        # Predicted as Normal by Stage 1 -> Final prediction is 0
        return 0
    else:
        # Predicted as Diseased by Stage 1 -> Proceed to Stage 2
        # Generate Stage 2 Meta Features
        meta_features_stage2_list = []
        with torch.no_grad():
            for cnn_name, cnn_classifier_model in cnn_base_classifiers_stage2_dict.items():
                # We already checked for None above, but double check for safety
                if cnn_classifier_model is None: continue
                #with torch.amp.autocast('cuda' if device_to_use == 'cuda' else 'cpu', enabled=torch.cuda.is_available()):
                outputs = cnn_classifier_model(image_tensor)
                probabilities = torch.softmax(outputs, dim=1) # 2 classes for Stage 2
                meta_features_stage2_list.append(probabilities.cpu().numpy())

        if not meta_features_stage2_list or len(meta_features_stage2_list) != len(cnn_base_classifiers_stage2_dict):
            print("错误: Stage 2 基础模型元特征生成失败。")
            # If Stage 2 meta-features fail, what should be the fallback?
            # Cannot differentiate between 1 and 2. Maybe return 1 as a default for diseased?
            return 1 # Fallback prediction if Stage 2 feature extraction fails
            
        image_meta_features_stage2 = np.concatenate(meta_features_stage2_list, axis=1)

        # Predict Stage 2 outcome (0: Observe, 1: Surgery)
        try:
            # meta_model_stage2_instance predicts 0 or 1 (mapped Stage 2 labels)
            predicted_stage2_mapped = meta_model_stage2_instance.predict(image_meta_features_stage2)[0]
            # Map Stage 2 prediction back to original labels (0->1, 1->2)
            final_prediction = 1 if predicted_stage2_mapped == 0 else 2
            return final_prediction
        except Exception as e:
            print(f"Stage 2 元模型预测时出错: {e}")
            # If Stage 2 meta-model prediction fails, fallback?
            return 1 # Fallback prediction if Stage 2 meta-model prediction fails


# --- Predict on Validation Directory ---
def predict_on_image_directory_two_stage(
    base_classifiers_stage1_loaded_dict, 
    meta_model_stage1_loaded_instance,
    base_classifiers_stage2_loaded_dict,
    meta_model_stage2_loaded_instance,
    directory_path, 
    image_transform_for_prediction, 
    device_for_prediction,
    output_excel_filename="Classification_Results_TwoStageStacking.xlsx"
    ):
    """
    在指定目录中的所有图像上运行两阶段 Stacking 模型预测，并将结果保存到 Excel。
    """
    if (not base_classifiers_stage1_loaded_dict or not all(base_classifiers_stage1_loaded_dict.values()) or 
        not meta_model_stage1_loaded_instance or 
        not base_classifiers_stage2_loaded_dict or not all(base_classifiers_stage2_loaded_dict.values()) or
        not meta_model_stage2_loaded_instance):
        print("错误: 部分或全部模型未成功加载，无法进行预测。")
        return

    # Find all supported image files (.png) in the directory
    image_paths = glob.glob(os.path.join(directory_path, "*.png")) 
    if not image_paths:
        print(f"警告: 在目录 {directory_path} 中未找到任何 .png 图像。")
        return

    prediction_results = []
    
    for img_path in tqdm(image_paths, desc="Predicting images with Two-Stage Stacking Ensemble", leave=True):
        try:
            predicted_label = predict_single_image_two_stage(
                base_classifiers_stage1_loaded_dict, 
                meta_model_stage1_loaded_instance,
                base_classifiers_stage2_loaded_dict,
                meta_model_stage2_loaded_instance,
                img_path, 
                image_transform_for_prediction, 
                device_for_prediction
            )
            
            if predicted_label is not None:
                # Extract image ID from filename (e.g., "0001.png" -> 1)
                base_name = os.path.splitext(os.path.basename(img_path))[0]
                try:
                    image_id = int(base_name)
                    prediction_results.append({"Image": image_id, "Pterygium": predicted_label})
                except ValueError:
                    tqdm.write(f"警告: 无法从文件名 {base_name} 解析图像ID。跳过此图像。")
            else:
                tqdm.write(f"图像 {img_path} 的预测失败或返回 None。")

        except Exception as e: # Catch any unexpected errors during the loop
            tqdm.write(f"处理或预测图像 {img_path} 时发生意外错误: {e}")
            # Optionally append a result with -1 or some indicator for failed prediction
            base_name = os.path.splitext(os.path.basename(img_path))[0]
            try:
                 image_id = int(base_name)
                 prediction_results.append({"Image": image_id, "Pterygium": -1}) # Use -1 to indicate failure
            except ValueError:
                 pass # Skip if ID cannot be parsed even for error reporting

    if not prediction_results:
        print("没有图像被成功预测。未生成结果文件。")
        return

    # Sort results by Image ID
    prediction_results.sort(key=lambda x: x["Image"])
    
    # Convert results to Pandas DataFrame and save to Excel
    results_df = pd.DataFrame(prediction_results, columns=["Image", "Pterygium"])
    
    try:
        results_df.to_excel(output_excel_filename, index=False)
        print(f"\n分类结果已保存到 {output_excel_filename}")
    except Exception as e:
        print(f"错误: 保存 Excel 文件失败: {e}")


# --- Call the prediction function on the validation directory ---
if all_base_stage1_loaded and final_meta_model_stage1_loaded and all_base_stage2_loaded and final_meta_model_stage2_loaded:
    print(f"\n开始对验证集目录 {val_image_dir} 中的图像进行预测...")
    # Use validation transform (val_transform) for prediction
    predict_on_image_directory_two_stage(
        final_cnn_classifiers_stage1_loaded, # Stage 1 base models
        final_meta_model_stage1_loaded,     # Stage 1 meta model
        final_cnn_classifiers_stage2_loaded, # Stage 2 base models
        final_meta_model_stage2_loaded,     # Stage 2 meta model
        val_image_dir,                      # Directory containing images to predict
        val_transform,                      # Image preprocessing transform
        device,                             # 'cuda' or 'cpu'
        output_excel_filename=f"Classification_Results_TwoStageStacking_{META_MODEL_TYPE.replace(' ', '_')}.xlsx" 
    )
else:
    print("错误：最终模型未能完全加载。跳过在验证目录上的预测。")